<a href="https://colab.research.google.com/github/S-EGK/Contollers/blob/main/Non-Linear%20Controllers/Robust%20Adaptive%20Controller%20for%20Payload%20Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
# importing libraries
from math import cos, sin, sqrt
import numpy as np
from random import random
import scipy.signal as signal
import scipy.linalg as linalg
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Circle
from IPython.display import HTML

# Parameters

## States

### Escort

In [ ]:
x1 = 0  # pitch
x2 = 0  # pitch rate
x3 = 0  # roll
x4 = 0  # roll rate
x5 = 0  # yaw
x6 = 0  # yaw rate
x7 = 0  # x
x8 = 0  # x rate
x9 = 0  # y
x10 = 0 # y rate
x11 = 0 # z
x12 = 0 # z rate

states = np.array([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12])

### Payload

In [ ]:
p1 = 0
p2 = 0
p3 = 0
p4 = 0
p5 = 0
p6 = 0
p7 = -5
p8 = 0
p9 = 0
p10 = 0
p11 = 0.1
p12 = 0

pstates = np.array([p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12])

p5_hat = 0
p6_hat = 0
p7_hat = -5
p8_hat = 0
p9_hat = 0
p10_hat = 0
p11_hat = 0.1
p12_hat = 0

pstates_hat = np.array([[p5_hat, p6_hat, p7_hat, p8_hat, p9_hat, p10_hat, p11_hat, p12_hat]]).T

P_ekf = np.eye(8)
Q_ekf = 0.1*np.eye(8)
R_ekf = np.array([[0.1]])

### Ship

In [ ]:
s1 = 0
s2 = 0
s3 = 0
s4 = 0
s5 = 0
s6 = 0
s7 = 5
s8 = 0
s9 = 0
s10 = 0
s11 = 0
s12 = 0

sstates = np.array([s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12])

ss7 = -5

ssstates = np.array([s1, s2, s3, s4, s5, s6, ss7, s8, s9, s10, s11, s12])

## Estimated Parameters

In [ ]:
a1_hat = 0
a2_hat = 0
a3_hat = 0
a4_hat = 0
a5_hat = 0
a6_hat = 0
a7_hat = 0
a8_hat = 0
a9_hat = 0
a10_hat = 0
a11_hat = 0

estimates = np.array([a1_hat, a2_hat, a3_hat, a4_hat, a5_hat, a6_hat, a7_hat, a8_hat, a9_hat, a10_hat, a11_hat])
pestimates = np.array([a1_hat, a2_hat, a3_hat, a4_hat, a5_hat, a6_hat, a7_hat, a8_hat, a9_hat, a10_hat, a11_hat])
sestimates = np.array([a1_hat, a2_hat, a3_hat, a4_hat, a5_hat, a6_hat, a7_hat, a8_hat, a9_hat, a10_hat, a11_hat])
ssestimates = np.array([a1_hat, a2_hat, a3_hat, a4_hat, a5_hat, a6_hat, a7_hat, a8_hat, a9_hat, a10_hat, a11_hat])

## Gains

In [ ]:
# Pitch Gains
k1 = 0.9
k2 = 0.9
Y1 = 0.9
Y2 = 0.9
Y3 = 0.9
# Roll Gains
k3 = 1.5
k4 = 1.5
Y4 = 0.9
Y5 = 0.9
Y6 = 0.9
# Yaw Gains
k5 = 10
k6 = 5
Y7 = 0.9
Y8 = 0.9
# X Gains
k7 = 0.9
k8 = 0.9
Y9 = 0.1
# Y Gains
k9 = 0.9
k10 = 0.9
Y10 = 0.1
# Z Gains
k11 = 0.9
k12 = 0.9
Y11 = 0.9

## Initial Inputs

In [ ]:
U1 = 0
U2 = 0
U3 = 0
U4 = 0
pU1 = 0
pU2 = 0
pU3 = 0
pU4 = 0
sU1 = 0
sU2 = 0
sU3 = 0
sU4 = 0
ssU1 = 0
ssU2 = 0
ssU3 = 0
ssU4 = 0
# Rotor Speeds
w1 = 0
w2 = 0
w3 = 0
w4 = 0

## Constants

In [ ]:
dx = 0.25
dy = 0.25
dz = 0.25
dph = 0.2
dth = 0.2
dps = 0.2
m = 10
g = 9.8
Jx = 0.114700 #moi wrt x
Jy = 0.057600 #moi wrt y
Jz = 0.171200 #moi wrt z
Jr = sqrt(Jx*Jx+Jy*Jy+Jz*Jz)
l = 0.5
k = 0.3

# Trajectory Stuff
a = 5
b = a
h1 = 10
h2 = 7.5

# Ship Parameters
x1s = 5
x2s = -5
hs = 0

## Normalized Parameters

In [ ]:
a1 = (Jy-Jz)/Jx
a2 = Jr/Jx
a3 = dph/Jx
a4 = (Jz-Jx)/Jy
a5 = Jr/Jy
a6 = dth/Jy
a7 = (Jx-Jy)/Jz
a8 = dps/Jz
a9 = dx/m
a10 = dy/m
a11 = dz/m

## Simulation Parameters

In [ ]:
dt = 0.1
Tfinal = 150
Ts = dt
Tsteps = Tfinal/Ts # number of frames
frames = int(Tsteps)

## Data Collectors

In [ ]:
x1_data = []
x2_data = []
x3_data = []
x4_data = []
x5_data = []
x6_data = []
x7_data = []
x8_data = []
x9_data = []
x10_data = []
x11_data = []
x12_data = []

a1_hat_data = []
a2_hat_data = []
a3_hat_data = []
a4_hat_data = []
a5_hat_data = []
a6_hat_data = []
a7_hat_data = []
a8_hat_data = []
a9_hat_data = []
a10_hat_data = []
a11_hat_data = []

a1_data = []
a2_data = []
a3_data = []
a4_data = []
a5_data = []
a6_data = []
a7_data = []
a8_data = []
a9_data = []
a10_data = []
a11_data = []

x_r_data = []
y_r_data = []
z_r_data = []
psi_r_data = []

x1d_data = []
x3d_data = []

U1_data = []
U2_data = []
U3_data = []
U4_data = []

w1_data = []
w2_data = []
w3_data = []
w4_data = []

p1_data = []
p2_data = []
p3_data = []
p4_data = []
p5_data = []
p6_data = []
p7_data = []
p8_data = []
p9_data = []
p10_data = []
p11_data = []
p12_data = []

pa1_hat_data = []
pa2_hat_data = []
pa3_hat_data = []
pa4_hat_data = []
pa5_hat_data = []
pa6_hat_data = []
pa7_hat_data = []
pa8_hat_data = []
pa9_hat_data = []
pa10_hat_data = []
pa11_hat_data = []

px_r_data = []
py_r_data = []
pz_r_data = []
ppsi_r_data = []

px1d_data = []
px3d_data = []

pU1_data = []
pU2_data = []
pU3_data = []
pU4_data = []

pw1_data = []
pw2_data = []
pw3_data = []
pw4_data = []

p5_hat_data = []
p6_hat_data = []
p7_hat_data = []
p8_hat_data = []
p9_hat_data = []
p10_hat_data = []
p11_hat_data = []
p12_hat_data = []

P_ekf_p5 = []
P_ekf_p7 = []
P_ekf_p9 = []
P_ekf_p11 = []

NP_ekf_p5 = []
NP_ekf_p7 = []
NP_ekf_p9 = []
NP_ekf_p11 = []

p5_err = []
p7_err = []
p9_err = []
p11_err = []

# Transformation Matrices

In [ ]:
# Rotational Matrix to convert body frame to inertial frame
def rot_mat_bod2veh(phi, theta, psi):
    R_b_v=np.array([[cos(theta)*cos(psi), sin(phi)*sin(theta)*cos(psi)-cos(phi)*sin(psi), cos(phi)*sin(theta)*cos(psi)+sin(phi)*sin(psi)],
                    [cos(theta)*sin(psi), sin(phi)*sin(theta)*sin(psi)+cos(phi)*cos(psi), cos(phi)*sin(theta)*sin(psi)-sin(phi)*cos(psi)],
                    [-sin(theta),         sin(phi)*cos(theta),                            cos(phi)*cos(theta)]])
    return R_b_v

In [ ]:
# Matrix to orient ned to xyz
def axes_ned2yxz():
  A_ned = [[1, 0, 0],
           [0, 1, 0],
           [0, 0, -1]]
  return A_ned

# Escort Model

## Reference Trajectory

In [ ]:
def ref_traj(t, h):
  x_r = a*np.cos(t)
  x_r_dot = -a*np.sin(t)
  x_r_ddot = -a*np.cos(t)
  x_ref = np.array([x_r, x_r_dot, x_r_ddot])

  y_r = b*np.sin(t)
  y_r_dot = b*np.cos(t)
  y_r_ddot = -b*np.sin(t)
  y_ref = np.array([y_r, y_r_dot, y_r_ddot])

  z_r = h
  z_r_dot = 0
  z_r_ddot = 0
  z_ref = np.array([z_r, z_r_dot, z_r_ddot])

  psi_r = np.pi/4
  psi_r_dot = 0
  psi_r_ddot = 0
  psi_ref = np.array([psi_r, psi_r_dot, psi_r_ddot])

  return x_ref, y_ref, z_ref, psi_ref

## Position Control

In [ ]:
def pos_ctrl(states, estimates, x_ref, y_ref, z_ref):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  # Block for x-pos
  z7 = x_ref[0] - x7                    # x-pos error state
  v1 = x_ref[1] + k7*z7               # virtual control 1
  sp1 = v1 - x8                         # sliding-mode manifold
  z7_dot = -k7*z7 + sp1               # rate change of error state
  v1_dot = x_ref[2] + k7*z7_dot       # rate change of virtual control 1
  v2 = v1_dot + z7 + a9*x8 + k8*sp1   # virtual control 2
  v2_hat = v1_dot + z7 + a9_hat*x8 + k8*sp1   # virtual control 2

  # Block for y-pos
  z9 = y_ref[0] - x9                    # y-pos error state
  v3 = y_ref[1] + k9*z9               # virtual control 3
  sp2 = v3 - x10                        # sliding-mode manifold
  z9_dot = -k9*z9 + sp2               # rate change of error state
  v3_dot = y_ref[2] + k9*z9_dot       # rate change of virtual control 3
  v4 = v3_dot + z9 + a10*x10 + k10*sp2 # virtual control 4
  v4_hat = v3_dot + z9 + a10_hat*x10 + k10*sp2 # virtual control 4

  # Block for z-pos
  z11 = z_ref[0] - x11                        # y-pos error state
  v5 = z_ref[1] + k11*z11                    # virtual control 3
  sp3 = v5 - x12                              # sliding-mode manifold
  z11_dot = -k11*z11 + sp3                   # rate change of error state
  v5_dot = z_ref[2] + k11*z11_dot            # rate change of virtual control 2
  v6 = v5_dot + z11 + g + a11*x12 + k12*sp3  # virtual control 2
  v6_hat = v5_dot + z11 + g + a11_hat*x12 + k12*sp3  # virtual control 2

  return v2, v4, v6, v2_hat, v4_hat, v6_hat

## Arithmetic Inverter

In [ ]:
def art_inv(t, states, estimates, h):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  x1_d = np.arctan2(-0.707106781186547*a10_hat*x10 + 0.707106781186548*a9_hat*x8 - 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + 0.707106781186548*k7*(-a*sin(t) - x8) + 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 0.707106781186547*k9*(b*cos(t) - x10) - 0.707106781186548*x7 + 0.707106781186547*x9, 1.41421356237309*sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11))

  x1_d_dot = 1.41421356237309*sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(-0.707106781186548*a*k7*cos(t) + 0.707106781186547*b*k9*sin(t) - 0.707106781186547*k10*(b*k9*cos(t) - b*sin(t)) + 0.707106781186548*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)/(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2) + 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)

  x1_d_ddot = 1.41421356237309*sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(0.707106781186548*a*k7*sin(t) + 0.707106781186547*b*k9*cos(t) - 0.707106781186547*k10*(-b*k9*sin(t) - b*cos(t)) + 0.707106781186548*k8*(-a*k7*cos(t) + a*sin(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)/(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2) + 1.41421356237309*sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(-0.5*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9) - 0.5*(-2*a*k7*cos(t) + 2.0*b*k9*sin(t) - 2.0*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9))*(-0.707106781186548*a*k7*cos(t) + 0.707106781186547*b*k9*sin(t) - 0.707106781186547*k10*(b*k9*cos(t) - b*sin(t)) + 0.707106781186548*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)/(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)**2 + 0.353553390593274*(2*a*k7*sin(t) - 2*b*k9*cos(t) + 2*k10*(-b*k9*sin(t) - b*cos(t)) + 2*k8*(-a*k7*cos(t) + a*sin(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) + 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-a*k7*cos(t) - b*k9*sin(t) + k10*(b*k9*cos(t) - b*sin(t)) + k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) + 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-0.707106781186548*a*k7*cos(t) + 0.707106781186547*b*k9*sin(t) - 0.707106781186547*k10*(b*k9*cos(t) - b*sin(t)) + 0.707106781186548*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) + 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a*k7*cos(t) - 0.707106781186547*b*k9*sin(t) + 0.707106781186547*k10*(b*k9*cos(t) - b*sin(t)) - 0.707106781186548*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) + 0.353553390593274*(-0.5*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9) - 0.5*(-2*a*k7*cos(t) + 2.0*b*k9*sin(t) - 2.0*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9))*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)**2*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) - 0.0883883476483185*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))**2*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/((1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)**(3/2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**4)

  x1r = np.array([x1_d, x1_d_dot, x1_d_ddot])

  x3_d = np.arctan2((a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11), 1.41421356237309)

  x3_d_dot = 1.41421356237309*(-a*k7*cos(t) - b*k9*sin(t) + k10*(b*k9*cos(t) - b*sin(t)) + k8*(-a*k7*sin(t) - a*cos(t)))/((2.0 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11))

  x3_d_ddot = 1.41421356237309*(a*k7*sin(t) - b*k9*cos(t) + k10*(-b*k9*sin(t) - b*cos(t)) + k8*(-a*k7*cos(t) + a*sin(t)))/((2.0 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)) - 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-a*k7*cos(t) - b*k9*sin(t) + k10*(b*k9*cos(t) - b*sin(t)) + k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/((1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)**2*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**3)

  x3r = np.array([x3_d, x3_d_dot, x3_d_ddot])

  U4 = 1.0*(1.0 + 0.5*(a10*x10 + a9*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10*x10 + a9*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2

  return U4, x1r, x3r

## Attitude Contol

In [ ]:
def att_ctrl(states, estimates, x1r, x3r, psi_ref, w_bar):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  # Pitch Block
  e1 = x1-x1r[0]
  x2d = -k1*e1 + x1r[1]
  z1 = x2 - x2d
  U1 = - e1 - a1_hat*x4*x6 - a2_hat*w_bar*x4 + a3_hat*x2 - k1*x2 + k1*x1r[1] + x1r[2] - k2*z1

  # Roll Block
  e3 = x3 - x3r[0]
  x4d = x3r[1] - k3*e3
  z3 = x4 - x4d
  U2 = - e3 - a4_hat*x2*x6 - a5_hat*w_bar*x2 + a6_hat*x4 + x3r[2] - k4*z3 - k3*x4 + k3*x3r[1]

  # Yaw Block
  e5 = x5 - psi_ref[0]
  x6d = psi_ref[1] - k6*e5
  z5 = x6 - x6d
  U3 = - e5 - a7_hat*x2*x4 + a8_hat*x6 + psi_ref[2] - k5*x6 + k5*psi_ref[1] - k6*z5

  return U1, U2, U3

## Rotor Speeds

In [ ]:
def rot_speed(U1, U2, U3, U4):
  p1 = l*k/Jx
  p2 = l*k/Jy
  p3 = l*k/Jz
  p4 = l*k/m
  P = np.array([[0,   -p1, 0,   p1],
                [-p2, 0,   p2,  0],
                [-p3, p3,  -p3, p3],
                [p4,  p4,  p4,  p4]])

  U = np.array([[U1, U2, U3, U4]]).T
  P_inv = np.linalg.inv(P)
  w = np.matmul(P_inv, U)

  w1 = sqrt(w[0])
  w2 = sqrt(w[1])
  w3 = sqrt(w[2])
  w4 = sqrt(w[3])

  w_bar = w4 + w3 - w2 - w1

  # w_bar = 1

  w1_data.append(w1)
  w2_data.append(w2)
  w3_data.append(w3)
  w4_data.append(w4)

  return w_bar

## Quad Dynamics

In [ ]:
def quad_dyna(t, y, w_bar, U1, U2, U3, v2, v4, v6):
  x1 = y[0]
  x2 = y[1]
  x3 = y[2]
  x4 = y[3]
  x5 = y[4]
  x6 = y[5]
  x7 = y[6]
  x8 = y[7]
  x9 = y[8]
  x10 = y[9]
  x11 = y[10]
  x12 = y[11]

  Cx1 = cos(x1)
  Sx1 = sin(x1)
  Cx3 = cos(x3)
  Sx3 = sin(x3)
  Cx5 = cos(x5)
  Sx5 = sin(x5)

  # Pitch
  x1_dot = x2
  x2_dot = a1*x4*x6 + a2*w_bar*x4 - a3*x2 + U1
  # Roll
  x3_dot = x4
  x4_dot = a4*x2*x6 + a5*w_bar*x2 - a6*x4 + U2
  # Yaw
  x5_dot = x6
  x6_dot = a7*x2*x4 - a8*x6 + U3
  # X
  x7_dot = x8
  x8_dot = v2 - a9*x8
  # Y
  x9_dot = x10
  x10_dot = v4 - a10*x10
  # Z
  x11_dot = x12
  x12_dot = - g + v6 - a11*x12

  return x1_dot, x2_dot, x3_dot, x4_dot, x5_dot, x6_dot, x7_dot, x8_dot, x9_dot, x10_dot, x11_dot, x12_dot

## Estimates Dynamics

In [ ]:
def est_dyna(t, y, states, x_ref, y_ref, z_ref, x1r, x3r, psi_ref, w_bar):
  a1_hat = y[0]
  a2_hat = y[1]
  a3_hat = y[2]
  a4_hat = y[3]
  a5_hat = y[4]
  a6_hat = y[5]
  a7_hat = y[6]
  a8_hat = y[7]
  a9_hat = y[8]
  a10_hat = y[9]
  a11_hat = y[10]
  
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  # Pitch
  e1 = x1-x1r[0]
  x2d = -k1*e1 + x1r[1]
  z1 = x2 - x2d
  a1_hat_dot = Y1*z1*x4*x6
  a2_hat_dot = Y2*z1*w_bar*x4
  a3_hat_dot = -Y3*z1*x2
  # Roll
  e3 = x3 - x3r[0]
  x4d = x3r[1] - k3*e3
  z3 = x4 - x4d
  a4_hat_dot = Y4*z3*x2*x6
  a5_hat_dot = Y5*z3*w_bar*x2
  a6_hat_dot = -Y6*z3*x4
  # Yaw
  e5 = x5 - psi_ref[0]
  x6d = psi_ref[1] - k6*e5
  z5 = x6 - x6d
  a7_hat_dot = Y7*z5*x2*x4
  a8_hat_dot = -Y8*z5*x6
  # X
  z7 = x_ref[0] - x7                    # x-pos error state
  v1 = x_ref[1] + k7*z7               # virtual control 1
  sp1 = v1 - x8                         # sliding-mode manifold
  a9_hat_dot = -Y9*sp1*x8
  # Y
  z9 = y_ref[0] - x9                    # y-pos error state
  v3 = y_ref[1] + k9*z9               # virtual control 3
  sp2 = v3 - x10                        # sliding-mode manifold
  a10_hat_dot = -Y10*sp2*x10
  # Z
  z11 = z_ref[0] - x11                        # y-pos error state
  v5 = z_ref[1] + k11*z11                    # virtual control 3
  sp3 = v5 - x12                              # sliding-mode manifold
  a11_hat_dot = -Y11*sp3*x12

  return a1_hat_dot, a2_hat_dot, a3_hat_dot, a4_hat_dot, a5_hat_dot, a6_hat_dot, a7_hat_dot, a8_hat_dot, a9_hat_dot, a10_hat_dot, a11_hat_dot

## Solver

In [ ]:
def step(states, estimates, i, U1, U2, U3, U4, h):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  x_ref, y_ref, z_ref, psi_ref = ref_traj(i/10, h)

  w_bar = rot_speed(U1, U2, U3, U4)

  v1, v2, v3, v2_hat, v4_hat, v6_hat = pos_ctrl(states, estimates, x_ref, y_ref, z_ref)

  U4, x1r, x3r = art_inv(i/10, states, estimates, h)

  U1, U2, U3 = att_ctrl(states, estimates, x1r, x3r, psi_ref, w_bar)

  sol = solve_ivp(lambda t, y: est_dyna(t, y, states, x_ref, y_ref, z_ref, x1r, x3r, psi_ref, w_bar), [0, Ts], [a1_hat, a2_hat, a3_hat, a4_hat, a5_hat, a6_hat, a7_hat, a8_hat, a9_hat, a10_hat, a11_hat], t_eval=np.linspace( 0, Ts, 10))
  estimates = sol.y[:,-1].T

  sol = solve_ivp(lambda t, y: quad_dyna(t, y, w_bar, U1, U2, U3, v1, v2, v3), [0, Ts], [x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12], t_eval=np.linspace( 0, Ts, 10))
  states = sol.y[:,-1].T

  return estimates, states, U1, U2, U3, U4, x_ref, y_ref, z_ref, x1r, x3r, psi_ref

# Payload Model

## Reference Trajectory

In [ ]:
def pref_traj(t, h):

  x_r = a*np.cos(t)
  x_r_dot = -a*np.sin(t)
  x_r_ddot = -a*np.cos(t)
  x_ref = np.array([x_r, x_r_dot, x_r_ddot])

  y_r = b*np.sin(t)
  y_r_dot = b*np.cos(t)
  y_r_ddot = -b*np.sin(t)
  y_ref = np.array([y_r, y_r_dot, y_r_ddot])

  z_r = h
  z_r_dot = 0
  z_r_ddot = 0
  z_ref = np.array([z_r, z_r_dot, z_r_ddot])

  psi_r = np.pi/4
  psi_r_dot = 0
  psi_r_ddot = 0
  psi_ref = np.array([psi_r, psi_r_dot, psi_r_ddot])

  return x_ref, y_ref, z_ref, psi_ref

## Position Control

In [ ]:
def ppos_ctrl(states, estimates, x_ref, y_ref, z_ref):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  # Block for x-pos
  z7 = x_ref[0] - x7                    # x-pos error state
  v1 = x_ref[1] + k7*z7               # virtual control 1
  sp1 = v1 - x8                         # sliding-mode manifold
  z7_dot = -k7*z7 + sp1               # rate change of error state
  v1_dot = x_ref[2] + k7*z7_dot       # rate change of virtual control 1
  v2 = v1_dot + z7 + a9*x8 + k8*sp1   # virtual control 2
  v2_hat = v1_dot + z7 + a9_hat*x8 + k8*sp1   # virtual control 2

  # Block for y-pos
  z9 = y_ref[0] - x9                    # y-pos error state
  v3 = y_ref[1] + k9*z9               # virtual control 3
  sp2 = v3 - x10                        # sliding-mode manifold
  z9_dot = -k9*z9 + sp2               # rate change of error state
  v3_dot = y_ref[2] + k9*z9_dot       # rate change of virtual control 3
  v4 = v3_dot + z9 + a10*x10 + k10*sp2 # virtual control 4
  v4_hat = v3_dot + z9 + a10_hat*x10 + k10*sp2 # virtual control 4

  # Block for z-pos
  z11 = z_ref[0] - x11                        # y-pos error state
  v5 = z_ref[1] + k11*z11                    # virtual control 3
  sp3 = v5 - x12                              # sliding-mode manifold
  z11_dot = -k11*z11 + sp3                   # rate change of error state
  v5_dot = z_ref[2] + k11*z11_dot            # rate change of virtual control 2
  v6 = v5_dot + z11 + g + a11*x12 + k12*sp3  # virtual control 2
  v6_hat = v5_dot + z11 + g + a11_hat*x12 + k12*sp3  # virtual control 2

  return v2, v4, v6, v2_hat, v4_hat, v6_hat

## Arithmetic Inverter

In [ ]:
def part_inv(t, states, estimates, h):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  x1_d = np.arctan2(-0.707106781186547*a10_hat*x10 + 0.707106781186548*a9_hat*x8 - 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + 0.707106781186548*k7*(-a*sin(t) - x8) + 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 0.707106781186547*k9*(b*cos(t) - x10) - 0.707106781186548*x7 + 0.707106781186547*x9, 1.41421356237309*sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11))

  x1_d_dot = 1.41421356237309*sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(-0.707106781186548*a*k7*cos(t) + 0.707106781186547*b*k9*sin(t) - 0.707106781186547*k10*(b*k9*cos(t) - b*sin(t)) + 0.707106781186548*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)/(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2) + 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)

  x1_d_ddot = 1.41421356237309*sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(0.707106781186548*a*k7*sin(t) + 0.707106781186547*b*k9*cos(t) - 0.707106781186547*k10*(-b*k9*sin(t) - b*cos(t)) + 0.707106781186548*k8*(-a*k7*cos(t) + a*sin(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)/(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2) + 1.41421356237309*sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(-0.5*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9) - 0.5*(-2*a*k7*cos(t) + 2.0*b*k9*sin(t) - 2.0*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9))*(-0.707106781186548*a*k7*cos(t) + 0.707106781186547*b*k9*sin(t) - 0.707106781186547*k10*(b*k9*cos(t) - b*sin(t)) + 0.707106781186548*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)/(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)**2 + 0.353553390593274*(2*a*k7*sin(t) - 2*b*k9*cos(t) + 2*k10*(-b*k9*sin(t) - b*cos(t)) + 2*k8*(-a*k7*cos(t) + a*sin(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) + 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-a*k7*cos(t) - b*k9*sin(t) + k10*(b*k9*cos(t) - b*sin(t)) + k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) + 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-0.707106781186548*a*k7*cos(t) + 0.707106781186547*b*k9*sin(t) - 0.707106781186547*k10*(b*k9*cos(t) - b*sin(t)) + 0.707106781186548*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) + 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a*k7*cos(t) - 0.707106781186547*b*k9*sin(t) + 0.707106781186547*k10*(b*k9*cos(t) - b*sin(t)) - 0.707106781186548*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) + 0.353553390593274*(-0.5*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9) - 0.5*(-2*a*k7*cos(t) + 2.0*b*k9*sin(t) - 2.0*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9))*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(sqrt(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)**2*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2) - 0.0883883476483185*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))**2*(0.707106781186548*a11_hat*x12 + 0.707106781186548*g + 0.707106781186548*h - 0.707106781186548*k11*x12 + 0.707106781186548*k12*(k11*(h - x11) - x12) - 0.707106781186548*x11)*(0.707106781186547*a10_hat*x10 - 0.707106781186548*a9_hat*x8 + 0.707106781186547*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) - 0.707106781186548*k7*(-a*sin(t) - x8) - 0.707106781186548*k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + 0.707106781186547*k9*(b*cos(t) - x10) + 0.707106781186548*x7 - 0.707106781186547*x9)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/((1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)**(3/2)*(1.0*(1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10_hat*x10 + a9_hat*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**4)

  x1r = np.array([x1_d, x1_d_dot, x1_d_ddot])

  x3_d = np.arctan2((a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11), 1.41421356237309)

  x3_d_dot = 1.41421356237309*(-a*k7*cos(t) - b*k9*sin(t) + k10*(b*k9*cos(t) - b*sin(t)) + k8*(-a*k7*sin(t) - a*cos(t)))/((2.0 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11))

  x3_d_ddot = 1.41421356237309*(a*k7*sin(t) - b*k9*cos(t) + k10*(-b*k9*sin(t) - b*cos(t)) + k8*(-a*k7*cos(t) + a*sin(t)))/((2.0 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)) - 0.353553390593274*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-a*k7*cos(t) - b*k9*sin(t) + k10*(b*k9*cos(t) - b*sin(t)) + k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)/((1 + 0.5*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)**2*(a11_hat*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**3)

  x3r = np.array([x3_d, x3_d_dot, x3_d_ddot])

  U4 = 1.0*(1.0 + 0.5*(a10*x10 + a9*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) + k9*(b*cos(t) - x10) - x7 - x9)**2/(a11*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2)*(a11*x12 + g + h - k11*x12 + k12*(k11*(h - x11) - x12) - x11)**2 + 0.5*(-1.0*a10*x10 + a9*x8 - 1.0*k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7) - x8) - 1.0*k9*(b*cos(t) - x10) - x7 + 1.0*x9)**2

  return U4, x1r, x3r

## Attitude Control

In [ ]:
def patt_ctrl(states, estimates, x1r, x3r, psi_ref, w_bar):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  # Pitch Block
  e1 = x1-x1r[0]
  x2d = -k1*e1 + x1r[1]
  z1 = x2 - x2d
  U1 = - e1 - a1_hat*x4*x6 - a2_hat*w_bar*x4 + a3_hat*x2 - k1*x2 + k1*x1r[1] + x1r[2] - k2*z1

  # Roll Block
  e3 = x3 - x3r[0]
  x4d = x3r[1] - k3*e3
  z3 = x4 - x4d
  U2 = - e3 - a4_hat*x2*x6 - a5_hat*w_bar*x2 + a6_hat*x4 + x3r[2] - k4*z3 - k3*x4 + k3*x3r[1]

  # Yaw Block
  e5 = x5 - psi_ref[0]
  x6d = psi_ref[1] - k6*e5
  z5 = x6 - x6d
  U3 = - e5 - a7_hat*x2*x4 + a8_hat*x6 + psi_ref[2] - k5*x6 + k5*psi_ref[1] - k6*z5

  return U1, U2, U3

## Rotor Speeds

In [ ]:
def prot_speed(U1, U2, U3, U4):
  p1 = l*k/Jx
  p2 = l*k/Jy
  p3 = l*k/Jz
  p4 = l*k/m
  P = np.array([[0,   -p1, 0,   p1],
                [-p2, 0,   p2,  0],
                [-p3, p3,  -p3, p3],
                [p4,  p4,  p4,  p4]])

  U = np.array([[U1, U2, U3, U4]]).T
  P_inv = np.linalg.inv(P)
  w = np.matmul(P_inv, U)

  w1 = sqrt(w[0])
  w2 = sqrt(w[1])
  w3 = sqrt(w[2])
  w4 = sqrt(w[3])

  w_bar = w4 + w3 - w2 - w1

  pw1_data.append(w1)
  pw2_data.append(w2)
  pw3_data.append(w3)
  pw4_data.append(w4)

  return w_bar

## Dynamics

In [ ]:
def pquad_dyna(t, y, w_bar, U1, U2, U3, v2, v4, v6):
  x1 = y[0]
  x2 = y[1]
  x3 = y[2]
  x4 = y[3]
  x5 = y[4]
  x6 = y[5]
  x7 = y[6]
  x8 = y[7]
  x9 = y[8]
  x10 = y[9]
  x11 = y[10]
  x12 = y[11]

  Cx1 = cos(x1)
  Sx1 = sin(x1)
  Cx3 = cos(x3)
  Sx3 = sin(x3)
  Cx5 = cos(x5)
  Sx5 = sin(x5)

  # Pitch
  x1_dot = x2
  x2_dot = a1*x4*x6 + a2*w_bar*x4 - a3*x2 + U1
  # Roll
  x3_dot = x4
  x4_dot = a4*x2*x6 + a5*w_bar*x2 - a6*x4 + U2
  # Yaw
  x5_dot = x6
  x6_dot = a7*x2*x4 - a8*x6 + U3
  # X
  x7_dot = x8
  x8_dot = v2 - a9*x8
  # Y
  x9_dot = x10
  x10_dot = v4 - a10*x10
  # Z
  x11_dot = x12
  x12_dot = - g + v6 - a11*x12

  return x1_dot, x2_dot, x3_dot, x4_dot, x5_dot, x6_dot, x7_dot, x8_dot, x9_dot, x10_dot, x11_dot, x12_dot

## Estimate Dynamics

In [ ]:
def pest_dyna(t, y, states, x_ref, y_ref, z_ref, x1r, x3r, psi_ref, w_bar):
  a1_hat = y[0]
  a2_hat = y[1]
  a3_hat = y[2]
  a4_hat = y[3]
  a5_hat = y[4]
  a6_hat = y[5]
  a7_hat = y[6]
  a8_hat = y[7]
  a9_hat = y[8]
  a10_hat = y[9]
  a11_hat = y[10]
  
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  # Pitch
  e1 = x1-x1r[0]
  x2d = -k1*e1 + x1r[1]
  z1 = x2 - x2d
  a1_hat_dot = Y1*z1*x4*x6
  a2_hat_dot = Y2*z1*w_bar*x4
  a3_hat_dot = -Y3*z1*x2
  # Roll
  e3 = x3 - x3r[0]
  x4d = x3r[1] - k3*e3
  z3 = x4 - x4d
  a4_hat_dot = Y4*z3*x2*x6
  a5_hat_dot = Y5*z3*w_bar*x2
  a6_hat_dot = -Y6*z3*x4
  # Yaw
  e5 = x5 - psi_ref[0]
  x6d = psi_ref[1] - k6*e5
  z5 = x6 - x6d
  a7_hat_dot = Y7*z5*x2*x4
  a8_hat_dot = -Y8*z5*x6
  # X
  z7 = x_ref[0] - x7                    # x-pos error state
  v1 = x_ref[1] + k7*z7               # virtual control 1
  sp1 = v1 - x8                         # sliding-mode manifold
  a9_hat_dot = -Y9*sp1*x8
  # Y
  z9 = y_ref[0] - x9                    # y-pos error state
  v3 = y_ref[1] + k9*z9               # virtual control 3
  sp2 = v3 - x10                        # sliding-mode manifold
  a10_hat_dot = -Y10*sp2*x10
  # Z
  z11 = z_ref[0] - x11                        # y-pos error state
  v5 = z_ref[1] + k11*z11                    # virtual control 3
  sp3 = v5 - x12                              # sliding-mode manifold
  a11_hat_dot = -Y11*sp3*x12

  return a1_hat_dot, a2_hat_dot, a3_hat_dot, a4_hat_dot, a5_hat_dot, a6_hat_dot, a7_hat_dot, a8_hat_dot, a9_hat_dot, a10_hat_dot, a11_hat_dot

## Estimator

In [ ]:
def pay_est(pstates, pstates_hat, P_ekf, states, sstates, ssstates, w_bar, U1, U2, U3, v2, v4, v6):
  [p5_hat] = pstates_hat[0]
  [p6_hat] = pstates_hat[1]
  [p7_hat] = pstates_hat[2]
  [p8_hat] = pstates_hat[3]
  [p9_hat] = pstates_hat[4]
  [p10_hat] = pstates_hat[5]
  [p11_hat] = pstates_hat[6]
  [p12_hat] = pstates_hat[7]

  p1 = pstates[0] + np.random.normal(0,0.1)
  p2 = pstates[1] + np.random.normal(0,0.1)  
  p3 = pstates[2] + np.random.normal(0,0.1)
  p4 = pstates[3] + np.random.normal(0,0.1)
  p5 = pstates[4] + np.random.normal(0,0.1)
  p6 = pstates[5] + np.random.normal(0,0.1)
  p7 = pstates[6] + np.random.normal(0,0.1)
  p8 = pstates[7] + np.random.normal(0,0.1)
  p9 = pstates[8] + np.random.normal(0,0.1)
  p10 = pstates[9] + np.random.normal(0,0.1)
  p11 = pstates[10] + np.random.normal(0,0.1)
  p12 = pstates[11] + np.random.normal(0,0.1)

  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  s1 = sstates[0]
  s2 = sstates[1]
  s3 = sstates[2]
  s4 = sstates[3]
  s5 = sstates[4]
  s6 = sstates[5]
  s7 = sstates[6]
  s8 = sstates[7]
  s9 = sstates[8]
  s10 = sstates[9]
  s11 = sstates[10]
  s12 = sstates[11]

  ss1 = ssstates[0]
  ss2 = ssstates[1]
  ss3 = ssstates[2]
  ss4 = ssstates[3]
  ss5 = ssstates[4]
  ss6 = ssstates[5]
  ss7 = ssstates[6]
  ss8 = ssstates[7]
  ss9 = ssstates[8]
  ss10 = ssstates[9]
  ss11 = ssstates[10]
  ss12 = ssstates[11]

  # # Pitch
  # p1_hat_dot = p2_hat
  # p2_hat_dot = a1*p4_hat*p6_hat + a2*w_bar*p4_hat - a3*p2_hat + U1
  # # Roll
  # p3_hat_dot = p4_hat
  # p4_hat_dot = a4*p2_hat*p6_hat + a5*w_bar*p2_hat - a6*p4_hat + U2
  # Yaw
  p5_hat_dot = p6_hat
  p6_hat_dot = a7*p2*p4 - a8*p6_hat + U3 + np.random.normal(0,0.1)
  # X
  p7_hat_dot = p8_hat
  p8_hat_dot = v2 + np.random.normal(0,0.1) - a9*p8_hat
  # Y
  p9_hat_dot = p10_hat
  p10_hat_dot = v4 + np.random.normal(0,0.1) - a10*p10_hat
  # Z
  p11_hat_dot = p12_hat
  p12_hat_dot = - g + v6 + np.random.normal(0,0.1) - a11*p12_hat

  pstates_hat_dot = np.array([[p5_hat_dot],
                              [p6_hat_dot],
                              [p7_hat_dot],
                              [p8_hat_dot],
                              [p9_hat_dot],
                              [p10_hat_dot],
                              [p11_hat_dot],
                              [p12_hat_dot]])
  
  pstates_hat = pstates_hat + Ts*pstates_hat_dot

  A_ekf = np.array([[0,1,0,0,0,0,0,0],
                    [0,-a8,0,0,0,0,0,0],
                    [0,0,0,1,0,0,0,0],
                    [0,0,0,-a9,0,0,0,0],
                    [0,0,0,0,0,1,0,0],
                    [0,0,0,0,0,-a10,0,0],
                    [0,0,0,0,0,0,0,1],
                    [0,0,0,0,0,0,0,-a11]])
  
  P_ekf_dot = np.matmul(A_ekf,P_ekf) + np.matmul(P_ekf, A_ekf.transpose()) + Q_ekf
  P_ekf = P_ekf + Ts*P_ekf_dot

  ran1 = ran_sen(pstates,states)
  rho1 = np.sqrt((p7_hat-x7)**2 + (p9_hat-x9)**2 + (p11_hat-x11)**2)
  C_ekf1 = np.array([[0, 0, (p7_hat-x7)/rho1, 0, (p9_hat-x9)/rho1, 0, (p11_hat-x11/rho1), 0]])
  var11 = np.matmul(C_ekf1, P_ekf)
  var21 = np.matmul(var11, np.transpose(C_ekf1))
  var31 = np.linalg.inv(np.array(R_ekf+var21))
  var41 = np.matmul(np.transpose(C_ekf1), var31)
  L_ekf1 = np.matmul(P_ekf, var41)
  var51 = ran1 - rho1
  pstates_hat = pstates_hat + L_ekf1*var51
  var61 = np.matmul(L_ekf1,C_ekf1)
  var71 = np.identity(8)-var61
  P_ekf = np.matmul(var71, P_ekf)

  ran2 = ran_sen(pstates,sstates)
  rho2 = np.sqrt((p7_hat-s7)**2 + (p9_hat-s9)**2 + (p11_hat-s11)**2)
  C_ekf2 = np.array([[0, 0, (p7_hat-s7)/rho2, 0, (p9_hat-s9)/rho2, 0, (p11_hat-s11/rho2), 0]])
  var12 = np.matmul(C_ekf2, P_ekf)
  var22 = np.matmul(var12, np.transpose(C_ekf2))
  var32 = np.linalg.inv(np.array(R_ekf+var22))
  var42 = np.matmul(np.transpose(C_ekf2), var32)
  L_ekf2 = np.matmul(P_ekf, var42)
  var52 = ran2 - rho2
  pstates_hat = pstates_hat + L_ekf2*var52
  var62 = np.matmul(L_ekf2,C_ekf2)
  var72 = np.identity(8)-var62
  P_ekf = np.matmul(var72, P_ekf)

  ran3 = ran_sen(pstates,ssstates)
  rho3 = np.sqrt((p7_hat-ss7)**2 + (p9_hat-ss9)**2 + (p11_hat-ss11)**2)
  C_ekf3 = np.array([[0, 0, (p7_hat-ss7)/rho3, 0, (p9_hat-ss9)/rho3, 0, (p11_hat-ss11/rho3), 0]])
  var13 = np.matmul(C_ekf3, P_ekf)
  var23 = np.matmul(var13, np.transpose(C_ekf3))
  var33 = np.linalg.inv(np.array(R_ekf+var23))
  var43 = np.matmul(np.transpose(C_ekf3), var33)
  L_ekf3 = np.matmul(P_ekf, var43)
  var53 = ran3 - rho3
  pstates_hat = pstates_hat + L_ekf3*var53
  var63 = np.matmul(L_ekf3,C_ekf3)
  var73 = np.identity(8)-var63
  P_ekf = np.matmul(var73, P_ekf)

  return pstates_hat, P_ekf

## Solver

In [ ]:
def pstep(states, estimates, i, U1, U2, U3, U4, h):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  if i < 750:
    if x11<0.9*h:
      h = 1.1*x11
    else:
      h = h
    x_ref, y_ref, z_ref, psi_ref = pref_traj(i/10, h)

    w_bar = prot_speed(U1, U2, U3, U4)

    v1, v2, v3, v2_hat, v4_hat, v6_hat = ppos_ctrl(states, estimates, x_ref, y_ref, z_ref)

    U4, x1r, x3r = part_inv(i/10, states, estimates, h)

    U1, U2, U3 = patt_ctrl(states, estimates, x1r, x3r, psi_ref, w_bar)
  elif i>=750:
    if x11<0.01:
      h = 0
    elif x11>0:
      h = 0.9*x11
    x_ref, y_ref, z_ref, psi_ref = sref_traj(i/10, h, x1s)

    w_bar = prot_speed(U1, U2, U3, U4)

    v1, v2, v3, v2_hat, v4_hat, v6_hat = spos_ctrl(states, estimates, x_ref, y_ref, z_ref)

    U4, x1r, x3r = sart_inv(i/10, states, estimates, h, x1s)

    U1, U2, U3 = satt_ctrl(states, estimates, x1r, x3r, psi_ref, w_bar)

  sol = solve_ivp(lambda t, y: pest_dyna(t, y, states, x_ref, y_ref, z_ref, x1r, x3r, psi_ref, w_bar), [0, Ts], [a1_hat, a2_hat, a3_hat, a4_hat, a5_hat, a6_hat, a7_hat, a8_hat, a9_hat, a10_hat, a11_hat], t_eval=np.linspace( 0, Ts, 10))
  estimates = sol.y[:,-1].T

  sol = solve_ivp(lambda t, y: pquad_dyna(t, y, w_bar, U1, U2, U3, v1, v2, v3), [0, Ts], [x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12], t_eval=np.linspace( 0, Ts, 10))
  states = sol.y[:,-1].T

  pU1_data.append(U1)
  pU2_data.append(U2)
  pU3_data.append(U3)
  pU4_data.append(U4)

  px_r_data.append(x_ref[0])
  py_r_data.append(y_ref[0])
  pz_r_data.append(z_ref[0])
  ppsi_r_data.append(psi_ref[0])

  px1d_data.append(x1r[0])
  px3d_data.append(x3r[0])

  return estimates, states, U1, U2, U3, U4, w_bar, v1, v2, v3

# Sensor

## Range Sensor

In [ ]:
def ran_sen(pstates,states):
  p1 = pstates[0]
  p2 = pstates[1]
  p3 = pstates[2]
  p4 = pstates[3]
  p5 = pstates[4]
  p6 = pstates[5]
  p7 = pstates[6]
  p8 = pstates[7]
  p9 = pstates[8]
  p10 = pstates[9]
  p11 = pstates[10]
  p12 = pstates[11]

  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  ran = np.sqrt((p7-x7)**2 + (p9-x9)**2 + (p11-x11)**2) + np.random.normal(0,0.1)

  return ran

# Ship Model

## Reference Trajectory

In [ ]:
def sref_traj(t, h, xs):

  x_r = xs + a*np.cos(t)
  x_r_dot = -a*np.sin(t)
  x_r_ddot = -a*np.cos(t)
  x_ref = np.array([x_r, x_r_dot, x_r_ddot])

  y_r = b*np.sin(t)
  y_r_dot = b*np.cos(t)
  y_r_ddot = -b*np.sin(t)
  y_ref = np.array([y_r, y_r_dot, y_r_ddot])

  z_r = h
  z_r_dot = 0
  z_r_ddot = 0
  z_ref = np.array([z_r, z_r_dot, z_r_ddot])

  psi_r = 0
  psi_r_dot = 0
  psi_r_ddot = 0
  psi_ref = np.array([psi_r, psi_r_dot, psi_r_ddot])

  return x_ref, y_ref, z_ref, psi_ref

## Position Control

In [ ]:
def spos_ctrl(states, estimates, x_ref, y_ref, z_ref):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  # Block for x-pos
  z7 = x_ref[0] - x7                    # x-pos error state
  v1 = x_ref[1] + k7*z7               # virtual control 1
  sp1 = v1 - x8                         # sliding-mode manifold
  z7_dot = -k7*z7 + sp1               # rate change of error state
  v1_dot = x_ref[2] + k7*z7_dot       # rate change of virtual control 1
  v2 = v1_dot + z7 + a9*x8 + k8*sp1   # virtual control 2
  v2_hat = v1_dot + z7 + a9_hat*x8 + k8*sp1   # virtual control 2

  # Block for y-pos
  z9 = y_ref[0] - x9                    # y-pos error state
  v3 = y_ref[1] + k9*z9               # virtual control 3
  sp2 = v3 - x10                        # sliding-mode manifold
  z9_dot = -k9*z9 + sp2               # rate change of error state
  v3_dot = y_ref[2] + k9*z9_dot       # rate change of virtual control 3
  v4 = v3_dot + z9 + a10*x10 + k10*sp2 # virtual control 4
  v4_hat = v3_dot + z9 + a10_hat*x10 + k10*sp2 # virtual control 4

  # Block for z-pos
  z11 = z_ref[0] - x11                        # y-pos error state
  v5 = z_ref[1] + k11*z11                    # virtual control 3
  sp3 = v5 - x12                              # sliding-mode manifold
  z11_dot = -k11*z11 + sp3                   # rate change of error state
  v5_dot = z_ref[2] + k11*z11_dot            # rate change of virtual control 2
  v6 = v5_dot + z11 + g + a11*x12 + k12*sp3  # virtual control 2
  v6_hat = v5_dot + z11 + g + a11_hat*x12 + k12*sp3  # virtual control 2

  return v2, v4, v6, v2_hat, v4_hat, v6_hat

## Arithmetic Inverter

In [ ]:
def sart_inv(t, states, estimates, h, xs):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  x1_d = np.arctan2(a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs, sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11))

  x1_d_dot = sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(-a*k7*cos(t) + k8*(-a*k7*sin(t) - a*cos(t)))*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)/((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2) + (-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-a9_hat*x8 - k7*(-a*sin(t) - x8) - k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + x7 - xs)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)/(2*sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11))

  x1_d_ddot = sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a*k7*sin(t) + k8*(-a*k7*cos(t) + a*sin(t)))*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)/((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2) + sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(-(-2*a*k7*cos(t) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs) - (-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs))*(-a*k7*cos(t) + k8*(-a*k7*sin(t) - a*cos(t)))*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)/((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2)**2 + (-a*k7*cos(t) + k8*(-a*k7*sin(t) - a*cos(t)))*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)/(2*sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)) + (a*k7*cos(t) - k8*(-a*k7*sin(t) - a*cos(t)))*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)/(2*sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)) + (2*a*k7*sin(t) - 2*b*k9*cos(t) + 2*k10*(-b*k9*sin(t) - b*cos(t)) + 2*k8*(-a*k7*cos(t) + a*sin(t)))*(-a9_hat*x8 - k7*(-a*sin(t) - x8) - k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + x7 - xs)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)/(2*sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)) + (-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-a*k7*cos(t) - b*k9*sin(t) + k10*(b*k9*cos(t) - b*sin(t)) + k8*(-a*k7*sin(t) - a*cos(t)))*(-a9_hat*x8 - k7*(-a*sin(t) - x8) - k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + x7 - xs)/(2*sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)) + (-(-2*a*k7*cos(t) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs) - (-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs))*(-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-a9_hat*x8 - k7*(-a*sin(t) - x8) - k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + x7 - xs)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)/(2*sqrt(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2)**2*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)) - (-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))**2*(-a9_hat*x8 - k7*(-a*sin(t) - x8) - k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + x7 - xs)*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(4*(1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)**(3/2)*((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9_hat*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**3)

  x1r = np.array([x1_d, x1_d_dot, x1_d_ddot])

  x3_d = np.arctan2((a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11), 1)

  x3_d_dot = (-a*k7*cos(t) - b*k9*sin(t) + k10*(b*k9*cos(t) - b*sin(t)) + k8*(-a*k7*sin(t) - a*cos(t)))/((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11))

  x3_d_ddot = (a*k7*sin(t) - b*k9*cos(t) + k10*(-b*k9*sin(t) - b*cos(t)) + k8*(-a*k7*cos(t) + a*sin(t)))/((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)) - (-2*a*k7*cos(t) - 2*b*k9*sin(t) + 2*k10*(b*k9*cos(t) - b*sin(t)) + 2*k8*(-a*k7*sin(t) - a*cos(t)))*(-a*k7*cos(t) - b*k9*sin(t) + k10*(b*k9*cos(t) - b*sin(t)) + k8*(-a*k7*sin(t) - a*cos(t)))*(a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)/((1 + (a10_hat*x10 + a9_hat*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)**2*(a11_hat*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**3)

  x3r = np.array([x3_d, x3_d_dot, x3_d_ddot])

  U4 = (1 + (a10*x10 + a9*x8 + k10*(b*cos(t) + k9*(b*sin(t) - x9) - x10) + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) + k9*(b*cos(t) - x10) - x7 - x9 + xs)**2/(a11*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2)*(a11*x12 + g - k11*x12 + k12*(-k11*x11 - x12) - x11)**2 + (a9*x8 + k7*(-a*sin(t) - x8) + k8*(-a*sin(t) + k7*(a*cos(t) - x7 + xs) - x8) - x7 + xs)**2

  return U4, x1r, x3r

## Attitude Control

In [ ]:
def satt_ctrl(states, estimates, x1r, x3r, psi_ref, w_bar):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  # Pitch Block
  e1 = x1-x1r[0]
  x2d = -k1*e1 + x1r[1]
  z1 = x2 - x2d
  U1 = - e1 - a1_hat*x4*x6 - a2_hat*w_bar*x4 + a3_hat*x2 - k1*x2 + k1*x1r[1] + x1r[2] - k2*z1

  # Roll Block
  e3 = x3 - x3r[0]
  x4d = x3r[1] - k3*e3
  z3 = x4 - x4d
  U2 = - e3 - a4_hat*x2*x6 - a5_hat*w_bar*x2 + a6_hat*x4 + x3r[2] - k4*z3 - k3*x4 + k3*x3r[1]

  # Yaw Block
  e5 = x5 - psi_ref[0]
  x6d = psi_ref[1] - k6*e5
  z5 = x6 - x6d
  U3 = - e5 - a7_hat*x2*x4 + a8_hat*x6 + psi_ref[2] - k5*x6 + k5*psi_ref[1] - k6*z5

  return U1, U2, U3

## Rotor Speeds

In [ ]:
def srot_speed(U1, U2, U3, U4):
  p1 = l*k/Jx
  p2 = l*k/Jy
  p3 = l*k/Jz
  p4 = l*k/m
  P = np.array([[0,   -p1, 0,   p1],
                [-p2, 0,   p2,  0],
                [-p3, p3,  -p3, p3],
                [p4,  p4,  p4,  p4]])

  U = np.array([[U1, U2, U3, U4]]).T
  P_inv = np.linalg.inv(P)
  w = np.matmul(P_inv, U)

  w1 = sqrt(w[0])
  w2 = sqrt(w[1])
  w3 = sqrt(w[2])
  w4 = sqrt(w[3])

  w_bar = w4 + w3 - w2 - w1

  return w_bar

## Dynamics

In [ ]:
def squad_dyna(t, y, w_bar, U1, U2, U3, v2, v4, v6):
  x1 = y[0]
  x2 = y[1]
  x3 = y[2]
  x4 = y[3]
  x5 = y[4]
  x6 = y[5]
  x7 = y[6]
  x8 = y[7]
  x9 = y[8]
  x10 = y[9]
  x11 = y[10]
  x12 = y[11]

  Cx1 = cos(x1)
  Sx1 = sin(x1)
  Cx3 = cos(x3)
  Sx3 = sin(x3)
  Cx5 = cos(x5)
  Sx5 = sin(x5)

  # Pitch
  x1_dot = x2
  x2_dot = a1*x4*x6 + a2*w_bar*x4 - a3*x2 + U1
  # Roll
  x3_dot = x4
  x4_dot = a4*x2*x6 + a5*w_bar*x2 - a6*x4 + U2
  # Yaw
  x5_dot = x6
  x6_dot = a7*x2*x4 - a8*x6 + U3
  # X
  x7_dot = x8
  x8_dot = v2 - a9*x8
  # Y
  x9_dot = x10
  x10_dot = v4 - a10*x10
  # Z
  x11_dot = x12
  x12_dot = - g + v6 - a11*x12

  return x1_dot, x2_dot, x3_dot, x4_dot, x5_dot, x6_dot, x7_dot, x8_dot, x9_dot, x10_dot, x11_dot, x12_dot

## Estimated Dynamics

In [ ]:
def sest_dyna(t, y, states, x_ref, y_ref, z_ref, x1r, x3r, psi_ref, w_bar):
  a1_hat = y[0]
  a2_hat = y[1]
  a3_hat = y[2]
  a4_hat = y[3]
  a5_hat = y[4]
  a6_hat = y[5]
  a7_hat = y[6]
  a8_hat = y[7]
  a9_hat = y[8]
  a10_hat = y[9]
  a11_hat = y[10]
  
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  # Pitch
  e1 = x1-x1r[0]
  x2d = -k1*e1 + x1r[1]
  z1 = x2 - x2d
  a1_hat_dot = Y1*z1*x4*x6
  a2_hat_dot = Y2*z1*w_bar*x4
  a3_hat_dot = -Y3*z1*x2
  # Roll
  e3 = x3 - x3r[0]
  x4d = x3r[1] - k3*e3
  z3 = x4 - x4d
  a4_hat_dot = Y4*z3*x2*x6
  a5_hat_dot = Y5*z3*w_bar*x2
  a6_hat_dot = -Y6*z3*x4
  # Yaw
  e5 = x5 - psi_ref[0]
  x6d = psi_ref[1] - k6*e5
  z5 = x6 - x6d
  a7_hat_dot = Y7*z5*x2*x4
  a8_hat_dot = -Y8*z5*x6
  # X
  z7 = x_ref[0] - x7                    # x-pos error state
  v1 = x_ref[1] + k7*z7               # virtual control 1
  sp1 = v1 - x8                         # sliding-mode manifold
  a9_hat_dot = -Y9*sp1*x8
  # Y
  z9 = y_ref[0] - x9                    # y-pos error state
  v3 = y_ref[1] + k9*z9               # virtual control 3
  sp2 = v3 - x10                        # sliding-mode manifold
  a10_hat_dot = -Y10*sp2*x10
  # Z
  z11 = z_ref[0] - x11                        # y-pos error state
  v5 = z_ref[1] + k11*z11                    # virtual control 3
  sp3 = v5 - x12                              # sliding-mode manifold
  a11_hat_dot = -Y11*sp3*x12

  return a1_hat_dot, a2_hat_dot, a3_hat_dot, a4_hat_dot, a5_hat_dot, a6_hat_dot, a7_hat_dot, a8_hat_dot, a9_hat_dot, a10_hat_dot, a11_hat_dot

## Solver

In [ ]:
def sstep(states, estimates, i, U1, U2, U3, U4, h, xs):
  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  x_ref, y_ref, z_ref, psi_ref = sref_traj(i/10, h, xs)

  w_bar = srot_speed(U1, U2, U3, U4)

  v1, v2, v3, v2_hat, v4_hat, v6_hat = spos_ctrl(states, estimates, x_ref, y_ref, z_ref)

  U4, x1r, x3r = sart_inv(i/10, states, estimates, h, xs)

  U1, U2, U3 = satt_ctrl(states, estimates, x1r, x3r, psi_ref, w_bar)

  sol = solve_ivp(lambda t, y: sest_dyna(t, y, states, x_ref, y_ref, z_ref, x1r, x3r, psi_ref, w_bar), [0, Ts], [a1_hat, a2_hat, a3_hat, a4_hat, a5_hat, a6_hat, a7_hat, a8_hat, a9_hat, a10_hat, a11_hat], t_eval=np.linspace( 0, Ts, 10))
  estimates = sol.y[:,-1].T

  sol = solve_ivp(lambda t, y: squad_dyna(t, y, w_bar, U1, U2, U3, v1, v2, v3), [0, Ts], [x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12], t_eval=np.linspace( 0, Ts, 10))
  states = sol.y[:,-1].T

  return estimates, states, U1, U2, U3, U4

# Simulation Setup

## Quad

In [ ]:
def build_quad(x7, x9, x11, x1, x3, x5):
  pn = x7
  pe = x9
  pd = -x11
  phi = x1
  theta = x3
  psi = x5
  
  # define width to use as reference to construct the quad 
  w = 1

  # defining the coordinates of the end points of the quad rotors
  p1 = np.array([w, 0, 0]).T
  p2 = np.array([-w, 0, 0]).T
  p3 = np.array([0, w, 0]).T
  p4 = np.array([0, -w, 0]).T

  # converting the quadrotor points to interial frame and translating as per ned
  R = rot_mat_bod2veh(phi, theta, psi) # rotational matrix
  pos_ned = np.array([pn, pe, pd]).T #
  p1r = np.matmul(R,p1)+pos_ned
  p2r = np.matmul(R,p2)+pos_ned
  p3r = np.matmul(R,p3)+pos_ned
  p4r = np.matmul(R,p4)+pos_ned

  # changing the orientation of ned wrt yxz
  A = axes_ned2yxz()
  p1ra = np.matmul(A,p1r)
  p2ra = np.matmul(A,p2r)
  p3ra = np.matmul(A,p3r)
  p4ra = np.matmul(A,p4r)

  # Rotor coordinates
  rot_x = [p1ra[0], p2ra[0], p3ra[0], p4ra[0]]
  rot_y = [p1ra[1], p2ra[1], p3ra[1], p4ra[1]]
  rot_z = [p1ra[2], p2ra[2], p3ra[2], p4ra[2]]

  # Arm coordinates
  arm1_x = [p1ra[0], p2ra[0]]
  arm1_y = [p1ra[1], p2ra[1]]
  arm1_z = [p1ra[2], p2ra[2]]

  arm2_x = [p3ra[0], p4ra[0]]
  arm2_y = [p3ra[1], p4ra[1]]
  arm2_z = [p3ra[2], p4ra[2]]

  # defining the coordinates of the payload
  v1 = np.array([-w/3,-w/3,w/2]).T
  v2 = np.array([-w/3,-w/3,0]).T
  v3 = np.array([w/3,-w/3,0]).T
  v4 = np.array([w/3,-w/3,w/2]).T
  v5 = np.array([w/3,w/3,w/2]).T
  v6 = np.array([w/3,w/3,0]).T
  v7 = np.array([-w/3,w/3,0]).T
  v8 = np.array([-w/3,w/3,w/2]).T

  # converting the payload points to interial frame and translating as per ned
  v1r = np.matmul(R,v1)+pos_ned
  v2r = np.matmul(R,v2)+pos_ned
  v3r = np.matmul(R,v3)+pos_ned
  v4r = np.matmul(R,v4)+pos_ned
  v5r = np.matmul(R,v5)+pos_ned
  v6r = np.matmul(R,v6)+pos_ned
  v7r = np.matmul(R,v7)+pos_ned
  v8r = np.matmul(R,v8)+pos_ned

  # changing the orientation of ned wrt yxz
  v1ra = np.matmul(A,v1r)
  v2ra = np.matmul(A,v2r)
  v3ra = np.matmul(A,v3r)
  v4ra = np.matmul(A,v4r)
  v5ra = np.matmul(A,v5r)
  v6ra = np.matmul(A,v6r)
  v7ra = np.matmul(A,v7r)
  v8ra = np.matmul(A,v8r)

  # defining the faces of the payload
  f1_x=[v1ra[0], v2ra[0], v3ra[0], v4ra[0], v1ra[0]]
  f1_y=[v1ra[1], v2ra[1], v3ra[1], v4ra[1], v1ra[1]]
  f1_z=[v1ra[2], v2ra[2], v3ra[2], v4ra[2], v1ra[2]]
  f2_x=[v5ra[0], v6ra[0], v7ra[0], v8ra[0], v5ra[0]]
  f2_y=[v5ra[1], v6ra[1], v7ra[1], v8ra[1], v5ra[1]]
  f2_z=[v5ra[2], v6ra[2], v7ra[2], v8ra[2], v5ra[2]]
  f3_x=[v4ra[0], v5ra[0], v6ra[0], v3ra[0], v4ra[0]]
  f3_y=[v4ra[1], v5ra[1], v6ra[1], v3ra[1], v4ra[1]]
  f3_z=[v4ra[2], v5ra[2], v6ra[2], v3ra[2], v4ra[2]]
  f4_x=[v1ra[0], v8ra[0], v7ra[0], v2ra[0], v1ra[0]]
  f4_y=[v1ra[1], v8ra[1], v7ra[1], v2ra[1], v1ra[1]]
  f4_z=[v1ra[2], v8ra[2], v7ra[2], v2ra[2], v1ra[2]]

  return rot_x, rot_y, rot_z, arm1_x, arm1_y, arm1_z, arm2_x, arm2_y, arm2_z, f1_x, f1_y, f1_z, f2_x, f2_y, f2_z, f3_x, f3_y, f3_z, f4_x, f4_y, f4_z

## Ship

In [ ]:
def build_ship(x7, x9, x11, x1, x3, x5, x):
  pn = x7
  pe = x9
  phi = x1
  theta = x3
  psi = x5

  # width and length to build the platform
  w = 0.5
  l = 1
  d = 0.1

  A = axes_ned2yxz()
  R = rot_mat_bod2veh(phi, theta, psi) # rotational matrix
  
  # defining the coordinates of the end points of the platform and translating to the correct position
  pos_ned = np.array([pn, pe, 0])
  v1 = np.array([l,-w,0]).T
  v2 = np.array([l,w,0]).T
  v3 = np.array([-l,w,0]).T
  v4 = np.array([-l,-w,0]).T
  v5 = np.array([l,-w,d]).T
  v6 = np.array([l,w,d]).T
  v7 = np.array([-l,w,d]).T
  v8 = np.array([-l,-w,d]).T

  # converting the payload points to interial frame and translating as per ned
  v1r = v1+pos_ned
  v2r = v2+pos_ned
  v3r = v3+pos_ned
  v4r = v4+pos_ned
  v5r = v5+pos_ned
  v6r = v6+pos_ned
  v7r = v7+pos_ned
  v8r = v8+pos_ned

  # changing the orientation of ned wrt yxz
  v1ra = np.matmul(A,v1r)
  v2ra = np.matmul(A,v2r)
  v3ra = np.matmul(A,v3r)
  v4ra = np.matmul(A,v4r)
  v5ra = np.matmul(A,v5r)
  v6ra = np.matmul(A,v6r)
  v7ra = np.matmul(A,v7r)
  v8ra = np.matmul(A,v8r)

  # defining the faces of the payload
  f1_x=[v1ra[0], v2ra[0], v3ra[0], v4ra[0], v1ra[0]]
  f1_y=[v1ra[1], v2ra[1], v3ra[1], v4ra[1], v1ra[1]]
  f1_z=[v1ra[2], v2ra[2], v3ra[2], v4ra[2], v1ra[2]]

  f2_x=[v5ra[0], v6ra[0], v7ra[0], v8ra[0], v5ra[0]]
  f2_y=[v5ra[1], v6ra[1], v7ra[1], v8ra[1], v5ra[1]]
  f2_z=[v5ra[2], v6ra[2], v7ra[2], v8ra[2], v5ra[2]]

  f3_x=[v1ra[0], v5ra[0], v8ra[0], v4ra[0], v1ra[0]]
  f3_y=[v1ra[1], v5ra[1], v8ra[1], v4ra[1], v1ra[1]]
  f3_z=[v1ra[2], v5ra[2], v8ra[2], v4ra[2], v1ra[2]]

  f4_x=[v2ra[0], v6ra[0], v7ra[0], v3ra[0], v2ra[0]]
  f4_y=[v2ra[1], v6ra[1], v7ra[1], v3ra[1], v2ra[1]]
  f4_z=[v2ra[2], v6ra[2], v7ra[2], v3ra[2], v2ra[2]]
    
  return f1_x, f1_y, f1_z, f2_x, f2_y, f2_z, f3_x, f3_y, f3_z, f4_x, f4_y, f4_z

In [ ]:
# Intial Plot
fig = plt.figure(figsize=(7,7), dpi=90)
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.set_xlim((-10, 10))            
ax.set_ylim((-10, 10))
ax.set_zlim((-10, 10))
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Simulation')

# Escort
rot_x, rot_y, rot_z, arm1_x, arm1_y, arm1_z, arm2_x, arm2_y, arm2_z, f1_x, f1_y, f1_z, f2_x, f2_y, f2_z, f3_x, f3_y, f3_z, f4_x, f4_y, f4_z = build_quad(x7, x9, x11, x1, x3, x5)

face1, = ax.plot(f1_x, f1_y, f1_z, 'g', lw=1.1)
face2, = ax.plot(f2_x, f2_y, f2_z, 'g', lw=1.1)
face3, = ax.plot(f3_x, f3_y, f3_z, 'g', lw=1.1)
face4, = ax.plot(f4_x, f4_y, f4_z, 'g', lw=1.1)

arm1, = ax.plot(arm1_x, arm1_y, arm1_z, 'k-', lw=1)
arm2, = ax.plot(arm2_x, arm2_y, arm2_z, 'k-', lw=1)
rotor, = ax.plot(rot_x, rot_y, rot_z, 'ro', ms = 2)

# Ship 1
s1f1_x, s1f1_y, s1f1_z, s1f2_x, s1f2_y, s1f2_z, s1f3_x, s1f3_y, s1f3_z, s1f4_x, s1f4_y, s1f4_z = build_ship(s7, s9, s11, s1, s3, s5, x1s)

s1_face1, = ax.plot(s1f1_x, s1f1_y, s1f1_z, 'b', lw=1)
s1_face2, = ax.plot(s1f2_x, s1f2_y, s1f2_z, 'b', lw=1)
s1_face3, = ax.plot(s1f3_x, s1f3_y, s1f3_z, 'b', lw=1)
s1_face4, = ax.plot(s1f4_x, s1f4_y, s1f4_z, 'b', lw=1)

# Ship 2
s2f1_x, s2f1_y, s2f1_z, s2f2_x, s2f2_y, s2f2_z, s2f3_x, s2f3_y, s2f3_z, s2f4_x, s2f4_y, s2f4_z = build_ship(s7, s9, s11, s1, s3, s5, x2s)

s2_face1, = ax.plot(s2f1_x, s2f1_y, s2f1_z, 'c', lw=1)
s2_face2, = ax.plot(s2f2_x, s2f2_y, s2f2_z, 'c', lw=1)
s2_face3, = ax.plot(s2f3_x, s2f3_y, s2f3_z, 'c', lw=1)
s2_face4, = ax.plot(s2f4_x, s2f4_y, s2f4_z, 'c', lw=1)

# Escort
prot_x, prot_y, prot_z, parm1_x, parm1_y, parm1_z, parm2_x, parm2_y, parm2_z, pf1_x, pf1_y, pf1_z, pf2_x, pf2_y, pf2_z, pf3_x, pf3_y, pf3_z, pf4_x, pf4_y, pf4_z = build_quad(p7, p9, p11, p1, p3, p5)

pface1, = ax.plot(pf1_x, pf1_y, pf1_z, 'y', lw=1)
pface2, = ax.plot(pf2_x, pf2_y, pf2_z, 'y', lw=1)
pface3, = ax.plot(pf3_x, pf3_y, pf3_z, 'y', lw=1)
pface4, = ax.plot(pf4_x, pf4_y, pf4_z, 'y', lw=1)

parm1, = ax.plot(parm1_x, parm1_y, parm1_z, 'k-', lw=1)
parm2, = ax.plot(parm2_x, parm2_y, parm2_z, 'k-', lw=1)
protor, = ax.plot(prot_x, prot_y, prot_z, 'ro', ms = 2)

## t = 0

# Simulation

In [ ]:
def drawframe(i):
  global states
  global estimates
  global U1
  global U2
  global U3
  global U4
  
  # Escort
  estimates, states, U1, U2, U3, U4, x_ref, y_ref, z_ref, x1r, x3r, psi_ref = step(states, estimates, i, U1, U2, U3, U4, h1)  

  x1 = states[0]
  x2 = states[1]
  x3 = states[2]
  x4 = states[3]
  x5 = states[4]
  x6 = states[5]
  x7 = states[6]
  x8 = states[7]
  x9 = states[8]
  x10 = states[9]
  x11 = states[10]
  x12 = states[11]

  a1_hat = estimates[0]
  a2_hat = estimates[1]
  a3_hat = estimates[2]
  a4_hat = estimates[3]
  a5_hat = estimates[4]
  a6_hat = estimates[5]
  a7_hat = estimates[6]
  a8_hat = estimates[7]
  a9_hat = estimates[8]
  a10_hat = estimates[9]
  a11_hat = estimates[10]

  x1_data.append(x1)
  x2_data.append(x2)
  x3_data.append(x3)
  x4_data.append(x4)
  x5_data.append(x5)
  x6_data.append(x6)
  x7_data.append(x7)
  x8_data.append(x8)
  x9_data.append(x9)
  x10_data.append(x10)
  x11_data.append(x11)
  x12_data.append(x12)

  a1_hat_data.append(a1_hat)
  a2_hat_data.append(a2_hat)
  a3_hat_data.append(a3_hat)
  a4_hat_data.append(a4_hat)
  a5_hat_data.append(a5_hat)
  a6_hat_data.append(a6_hat)
  a7_hat_data.append(a7_hat)
  a8_hat_data.append(a8_hat)
  a9_hat_data.append(a9_hat)
  a10_hat_data.append(a10_hat)
  a11_hat_data.append(a11_hat)

  a1_data.append(a1)
  a2_data.append(a2)
  a3_data.append(a3)
  a4_data.append(a4)
  a5_data.append(a5)
  a6_data.append(a6)
  a7_data.append(a7)
  a8_data.append(a8)
  a9_data.append(a9)
  a10_data.append(a10)
  a11_data.append(a11)

  U1_data.append(U1)
  U2_data.append(U2)
  U3_data.append(U3)
  U4_data.append(U4)

  x_r_data.append(x_ref[0])
  y_r_data.append(y_ref[0])
  z_r_data.append(z_ref[0])
  psi_r_data.append(psi_ref[0])

  x1d_data.append(x1r[0])
  x3d_data.append(x3r[0])

  rot_x, rot_y, rot_z, arm1_x, arm1_y, arm1_z, arm2_x, arm2_y, arm2_z, f1_x, f1_y, f1_z, f2_x, f2_y, f2_z, f3_x, f3_y, f3_z, f4_x, f4_y, f4_z = build_quad(x7, x9, x11, x1, x3, x5)

  face1.set_data(f1_x, f1_y)
  face1.set_3d_properties(f1_z)
  face2.set_data(f2_x, f2_y)
  face2.set_3d_properties(f2_z)
  face3.set_data(f3_x, f3_y)
  face3.set_3d_properties(f3_z)
  face4.set_data(f4_x, f4_y)
  face4.set_3d_properties(f3_z)

  arm1.set_data(arm1_x, arm1_y)
  arm1.set_3d_properties(arm1_z)
  arm2.set_data(arm2_x, arm2_y)
  arm2.set_3d_properties(arm2_z)
  rotor.set_data(rot_x, rot_y)
  rotor.set_3d_properties(rot_z)  

  # Ship 1
  global sstates
  global sestimates
  global sU1
  global sU2
  global sU3
  global sU4
  sestimates, sstates, sU1, sU2, sU3, sU4 = sstep(sstates, sestimates, i, sU1, sU2, sU3, sU4, hs, x1s)

  s1 = sstates[0]
  s2 = sstates[1]
  s3 = sstates[2]
  s4 = sstates[3]
  s5 = sstates[4]
  s6 = sstates[5]
  s7 = sstates[6]
  s8 = sstates[7]
  s9 = sstates[8]
  s10 = sstates[9]
  s11 = sstates[10]
  s12 = sstates[11]

  s1_f1_x, s1_f1_y, s1_f1_z, s1_f2_x, s1_f2_y, s1_f2_z, s1_f3_x, s1_f3_y, s1_f3_z, s1_f4_x, s1_f4_y, s1_f4_z = build_ship(s7, s9, s11, s1, s3, s5, x1s)

  s1_face1.set_data(s1_f1_x, s1_f1_y)
  s1_face1.set_3d_properties(s1_f1_z)
  s1_face2.set_data(s1_f2_x, s1_f2_y)
  s1_face2.set_3d_properties(s1_f2_z)
  s1_face3.set_data(s1_f3_x, s1_f3_y)
  s1_face3.set_3d_properties(s1_f3_z)
  s1_face4.set_data(s1_f4_x, s1_f4_y)
  s1_face4.set_3d_properties(s1_f3_z)

  # Ship 2
  global ssstates
  global ssestimates
  global ssU1
  global ssU2
  global ssU3
  global ssU4

  ssestimates, ssstates, ssU1, ssU2, ssU3, ssU4 = sstep(ssstates, ssestimates, i, ssU1, ssU2, ssU3, ssU4, hs, x2s)

  ss1 = ssstates[0]
  ss2 = ssstates[1]
  ss3 = ssstates[2]
  ss4 = ssstates[3]
  ss5 = ssstates[4]
  ss6 = ssstates[5]
  ss7 = ssstates[6]
  ss8 = ssstates[7]
  ss9 = ssstates[8]
  ss10 = ssstates[9]
  ss11 = ssstates[10]
  ss12 = ssstates[11]

  s2_f1_x, s2_f1_y, s2_f1_z, s2_f2_x, s2_f2_y, s2_f2_z, s2_f3_x, s2_f3_y, s2_f3_z, s2_f4_x, s2_f4_y, s2_f4_z = build_ship(ss7, ss9, ss11, ss1, ss3, ss5, x2s)

  s2_face1.set_data(s2_f1_x, s2_f1_y)
  s2_face1.set_3d_properties(s2_f1_z)
  s2_face2.set_data(s2_f2_x, s2_f2_y)
  s2_face2.set_3d_properties(s2_f2_z)
  s2_face3.set_data(s2_f3_x, s2_f3_y)
  s2_face3.set_3d_properties(s2_f3_z)
  s2_face4.set_data(s2_f4_x, s2_f4_y)
  s2_face4.set_3d_properties(s2_f3_z)

  # Payload
  global pstates
  global pestimates
  global pU1
  global pU2
  global pU3
  global pU4

  pestimates, pstates, pU1, pU2, pU3, pU4, w_bar, v1, v2, v3 = pstep(pstates, pestimates, i, pU1, pU2, pU3, pU4, h2)

  p1 = pstates[0]
  p2 = pstates[1]
  p3 = pstates[2]
  p4 = pstates[3]
  p5 = pstates[4]
  p6 = pstates[5]
  p7 = pstates[6]
  p8 = pstates[7]
  p9 = pstates[8]
  p10 = pstates[9]
  p11 = pstates[10]
  p12 = pstates[11]

  pa1_hat = pestimates[0]
  pa2_hat = pestimates[1]
  pa3_hat = pestimates[2]
  pa4_hat = pestimates[3]
  pa5_hat = pestimates[4]
  pa6_hat = pestimates[5]
  pa7_hat = pestimates[6]
  pa8_hat = pestimates[7]
  pa9_hat = pestimates[8]
  pa10_hat = pestimates[9]
  pa11_hat = pestimates[10]

  pa1_hat_data.append(pa1_hat)
  pa2_hat_data.append(pa2_hat)
  pa3_hat_data.append(pa3_hat)
  pa4_hat_data.append(pa4_hat)
  pa5_hat_data.append(pa5_hat)
  pa6_hat_data.append(pa6_hat)
  pa7_hat_data.append(pa7_hat)
  pa8_hat_data.append(pa8_hat)
  pa9_hat_data.append(pa9_hat)
  pa10_hat_data.append(pa10_hat)
  pa11_hat_data.append(pa11_hat)

  p1_data.append(p1)
  p2_data.append(p2)
  p3_data.append(p3)
  p4_data.append(p4)
  p5_data.append(p5)
  p6_data.append(p6)
  p7_data.append(p7)
  p8_data.append(p8)
  p9_data.append(p9)
  p10_data.append(p10)
  p11_data.append(p11)
  p12_data.append(p12)

  prot_x, prot_y, prot_z, parm1_x, parm1_y, parm1_z, parm2_x, parm2_y, parm2_z, pf1_x, pf1_y, pf1_z, pf2_x, pf2_y, pf2_z, pf3_x, pf3_y, pf3_z, pf4_x, pf4_y, pf4_z = build_quad(p7, p9, p11, p1, p3, p5)

  pface1.set_data(pf1_x, pf1_y)
  pface1.set_3d_properties(pf1_z)
  pface2.set_data(pf2_x, pf2_y)
  pface2.set_3d_properties(pf2_z)
  pface3.set_data(pf3_x, pf3_y)
  pface3.set_3d_properties(pf3_z)
  pface4.set_data(pf4_x, pf4_y)
  pface4.set_3d_properties(pf3_z)

  parm1.set_data(parm1_x, parm1_y)
  parm1.set_3d_properties(parm1_z)
  parm2.set_data(parm2_x, parm2_y)
  parm2.set_3d_properties(parm2_z)
  protor.set_data(prot_x, prot_y)
  protor.set_3d_properties(prot_z)

  # Payload Estimation
  global pstates_hat
  global P_ekf

  pstates_hat, P_ekf = pay_est(pstates, pstates_hat, P_ekf, states, sstates, ssstates, w_bar, pU1, pU2, pU3, v1, v2, v3)

  p5_hat_data.append(pstates_hat[0])
  p6_hat_data.append(pstates_hat[1])
  p7_hat_data.append(pstates_hat[2])
  p8_hat_data.append(pstates_hat[3])
  p9_hat_data.append(pstates_hat[4])
  p10_hat_data.append(pstates_hat[5])
  p11_hat_data.append(pstates_hat[6])
  p12_hat_data.append(pstates_hat[7])

  P_ekf_p5.append(3*sqrt(P_ekf[0,0]))
  P_ekf_p7.append(3*sqrt(P_ekf[2,2]))
  P_ekf_p9.append(3*sqrt(P_ekf[4,4]))
  P_ekf_p11.append(3*sqrt(P_ekf[6,6]))

  NP_ekf_p5.append(-3*sqrt(P_ekf[0,0]))
  NP_ekf_p7.append(-3*sqrt(P_ekf[2,2]))
  NP_ekf_p9.append(-3*sqrt(P_ekf[4,4]))
  NP_ekf_p11.append(-3*sqrt(P_ekf[6,6]))

  p5_err.append(pstates_hat[0] - p5)
  p7_err.append(pstates_hat[2] - p7)
  p9_err.append(pstates_hat[4] - p9)
  p11_err.append(pstates_hat[6] - p11)

  return face1, face2, face3, face4, arm1, arm2, rotor, s1_face1, s1_face2, s1_face3, s1_face4, s2_face1, s2_face2, s2_face3, s2_face4, pface1, pface2, pface3, pface4, parm1, parm2, protor

In [ ]:
# Creating the Simulatiom
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Simulation')
# blit=True re-draws only the parts that have changed.
anim = animation.FuncAnimation(fig, drawframe, frames=frames, interval=100, blit=True)

## Video

In [ ]:
# Converting the animated result into a  html5 video for visualization
HTML(anim.to_html5_video())

# Plots

## Escort Trajectory

In [ ]:
fig0 = plt.figure(figsize=(30,30))
ax1 = fig0.add_subplot(4, 3, 1, projection='3d')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Trajectory')
pn1 = ax1.plot(x7_data,x9_data,x11_data, label = "Traj")
pn1 = ax1.plot(x_r_data,y_r_data,z_r_data, 'r:', label = "X_ref")
ax1.legend(loc = "best")

## Escort States

In [ ]:
fig1 = plt.figure(figsize=(15,20))
ax1 = fig1.add_subplot(4, 3, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('X')
ax1.set_title('X vs Time Step (b:true, r:ref)')
pn1 = ax1.plot(range(len(x7_data)),x7_data, label = "X")
pn1 = ax1.plot(range(len(x_r_data)),x_r_data, 'r:', label = "X_ref")
ax1.legend(loc = "best")

ax2 = fig1.add_subplot(4, 3, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('Y')
ax2.set_title('Y vs Time Step (b:true, r:ref)')
pn1 = ax2.plot(range(len(x9_data)),x9_data, label = "Y")
pn1 = ax2.plot(range(len(y_r_data)),y_r_data, 'r:', label = "Y_ref")
ax2.legend(loc = "best")

ax3 = fig1.add_subplot(4, 3, 3)
ax3.set_xlabel('Time Step')
ax3.set_ylabel('Z')
ax3.set_title('Z vs Time Step (b:true, r:ref)')
pn1 = ax3.plot(range(len(x11_data)),x11_data, label = "Z")
pn1 = ax3.plot(range(len(z_r_data)),z_r_data, 'r:', label = "Z_ref")
ax3.legend(loc = "best")

ax4 = fig1.add_subplot(4, 3, 4)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('Phi')
ax4.set_title('State Plot of Phi')
pn1 = ax4.plot(range(len(x1_data)),x1_data, label = "Phi")
pn1 = ax4.plot(range(len(x1d_data)),x1d_data, 'r:', label = "Phi_d")
ax4.legend(loc = "best")

ax5 = fig1.add_subplot(4, 3, 5)
ax5.set_xlabel('Time Step')
ax5.set_ylabel('Theta')
ax5.set_title('State Plot of Theta')
pn1 = ax5.plot(range(len(x3_data)),x3_data, label = "Theta")
pn1 = ax5.plot(range(len(x3d_data)),x3d_data, 'r:', label = "Theta_d")
ax5.legend(loc = "best")

ax6 = fig1.add_subplot(4, 3, 6)
ax6.set_xlabel('Time Step')
ax6.set_ylabel('Psi')
ax6.set_title('State Plot of Psi')
pn1 = ax6.plot(range(len(x5_data)),x5_data, label = "Psi")
pn1 = ax6.plot(range(len(psi_r_data)),psi_r_data, 'r:', label = "Psi_ref")
ax6.legend(loc = "best")

## Escort Inputs

In [ ]:
fig2 = plt.figure(figsize=(15,20))
ax1 = fig2.add_subplot(4, 2, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('U1')
ax1.set_title('U1 vs Time Step')
pn1 = ax1.plot(range(len(U1_data)),U1_data, label = "U1")
ax1.legend(loc = "best")

ax2 = fig2.add_subplot(4, 2, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('U2')
ax2.set_title('U2 vs Time Step')
pn1 = ax2.plot(range(len(U2_data)),U2_data, label = "U2")
ax2.legend(loc = "best")

ax3 = fig2.add_subplot(4, 2, 3)
ax3.set_xlabel('Time Step')
ax3.set_ylabel('U3')
ax3.set_title('U3 vs Time Step')
pn1 = ax3.plot(range(len(U3_data)),U3_data, label = "U3")
ax3.legend(loc = "best")

ax4 = fig2.add_subplot(4, 2, 4)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('U4')
ax4.set_title('U4 vs Time Step')
pn1 = ax4.plot(range(len(U4_data)),U4_data, label = "U4")
ax4.legend(loc = "best")

ax5 = fig2.add_subplot(4, 2, 5)
ax5.set_xlabel('Time Step')
ax5.set_ylabel('w1')
ax5.set_title('w1 vs Time Step')
pn1 = ax5.plot(range(len(w1_data)),w1_data, label = "w1")
ax5.legend(loc = "best")

ax6 = fig2.add_subplot(4, 2, 6)
ax6.set_xlabel('Time Step')
ax6.set_ylabel('w2')
ax6.set_title('w2 vs Time Step')
pn1 = ax6.plot(range(len(w2_data)),w2_data, label = "w2")
ax6.legend(loc = "best")

ax7 = fig2.add_subplot(4, 2, 7)
ax7.set_xlabel('Time Step')
ax7.set_ylabel('w3')
ax7.set_title('w3 vs Time Step')
pn1 = ax7.plot(range(len(w3_data)),w3_data, label = "w3")
ax7.legend(loc = "best")

ax8 = fig2.add_subplot(4, 2, 8)
ax8.set_xlabel('Time Step')
ax8.set_ylabel('w4')
ax8.set_title('w4 vs Time Step')
pn1 = ax8.plot(range(len(w4_data)),w4_data, label = "w4")
ax8.legend(loc = "best")

## Estimated Values

In [ ]:
fig3 = plt.figure(figsize=(17,17))
ax1 = fig3.add_subplot(3, 4, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('a1')
ax1.set_title('a1 vs Time Step')
pn1 = ax1.plot(range(len(a1_hat_data)),a1_hat_data, label = "a1_hat")
pn1 = ax1.plot(range(len(a1_data)),a1_data, 'r:', label = "a1")
ax1.legend(loc = "best")

ax2 = fig3.add_subplot(3, 4, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('a2')
ax2.set_title('a2 vs Time Step')
pn1 = ax2.plot(range(len(a2_hat_data)),a2_hat_data, label = "a2_hat")
pn1 = ax2.plot(range(len(a2_data)),a2_data, 'r:', label = "a2")
ax2.legend(loc = "best")

ax3 = fig3.add_subplot(3, 4, 3)
ax3.set_xlabel('Time Step')
ax3.set_ylabel('a3')
ax3.set_title('a3 vs Time Step')
pn1 = ax3.plot(range(len(a3_hat_data)),a3_hat_data, label = "a3_hat")
pn1 = ax3.plot(range(len(a3_data)),a3_data, 'r:', label = "a3")
ax3.legend(loc = "best")

ax4 = fig3.add_subplot(3, 4, 4)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('a4')
ax4.set_title('a4 vs Time Step')
pn1 = ax4.plot(range(len(a4_hat_data)),a4_hat_data, label = "a4_hat")
pn1 = ax4.plot(range(len(a4_data)),a4_data, 'r:', label = "a4")
ax4.legend(loc = "best")

ax5 = fig3.add_subplot(3, 4, 5)
ax5.set_xlabel('Time Step')
ax5.set_ylabel('a5')
ax5.set_title('a5 vs Time Step')
pn1 = ax5.plot(range(len(a5_hat_data)),a5_hat_data, label = "a5_hat")
pn1 = ax5.plot(range(len(a5_data)),a5_data, 'r:', label = "a5")
ax5.legend(loc = "best")

ax6 = fig3.add_subplot(3, 4, 6)
ax6.set_xlabel('Time Step')
ax6.set_ylabel('a6')
ax6.set_title('a6 vs Time Step')
pn1 = ax6.plot(range(len(a6_hat_data)),a6_hat_data, label = "a6_hat")
pn1 = ax6.plot(range(len(a6_data)),a6_data, 'r:', label = "a6")
ax6.legend(loc = "best")

ax7 = fig3.add_subplot(3, 4, 7)
ax7.set_xlabel('Time Step')
ax7.set_ylabel('a7')
ax7.set_title('a7 vs Time Step')
pn1 = ax7.plot(range(len(a7_hat_data)),a7_hat_data, label = "a7_hat")
pn1 = ax7.plot(range(len(a7_data)),a7_data, 'r:', label = "a7")
ax7.legend(loc = "best")

ax8 = fig3.add_subplot(3, 4, 8)
ax8.set_xlabel('Time Step')
ax8.set_ylabel('a8')
ax8.set_title('a8 vs Time Step')
pn1 = ax8.plot(range(len(a8_hat_data)),a8_hat_data, label = "a8_hat")
pn1 = ax8.plot(range(len(a8_data)),a8_data, 'r:', label = "a8")
ax8.legend(loc = "best")

ax9 = fig3.add_subplot(3, 4, 9)
ax9.set_xlabel('Time Step')
ax9.set_ylabel('a9')
ax9.set_title('a9 vs Time Step')
pn1 = ax9.plot(range(len(a9_hat_data)),a9_hat_data, label = "a9_hat")
pn1 = ax9.plot(range(len(a9_data)),a9_data, 'r:', label = "a9")
ax9.legend(loc = "best")

ax10 = fig3.add_subplot(3, 4, 10)
ax10.set_xlabel('Time Step')
ax10.set_ylabel('a10')
ax10.set_title('a10 vs Time Step')
pn1 = ax10.plot(range(len(a10_hat_data)),a10_hat_data, label = "a10_hat")
pn1 = ax10.plot(range(len(a10_data)),a10_data, 'r:', label = "a10")
ax10.legend(loc = "best")

ax11 = fig3.add_subplot(3, 4, 11)
ax11.set_xlabel('Time Step')
ax11.set_ylabel('a11')
ax11.set_title('a11 vs Time Step')
pn1 = ax11.plot(range(len(a11_hat_data)),a11_hat_data, label = "a11_hat")
pn1 = ax11.plot(range(len(a11_data)),a11_data, 'r:', label = "a11")
ax11.legend(loc = "best")

## Payload Trajectory

In [ ]:
fig4 = plt.figure(figsize=(30,30))
ax1 = fig4.add_subplot(4, 3, 1, projection='3d')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Trajectory')
pn1 = ax1.plot(p7_data,p9_data,p11_data, label = "Traj")
# pn1 = ax1.plot(x_r_data,y_r_data,z_r_data, 'r:', label = "X_ref")
ax1.legend(loc = "best")

## Payload States

In [ ]:
fig5 = plt.figure(figsize=(15,20))
ax1 = fig5.add_subplot(4, 3, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('X')
ax1.set_title('X vs Time Step (b:true, r:ref)')
pn1 = ax1.plot(range(len(p7_data)),p7_data, label = "X")
# pn1 = ax1.plot(range(len(p7_hat_data)),p7_hat_data, 'k:', label = "X_hat")
pn1 = ax1.plot(range(len(px_r_data)),px_r_data, 'r:', label = "X_ref")
ax1.legend(loc = "best")

ax2 = fig5.add_subplot(4, 3, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('Y')
ax2.set_title('Y vs Time Step (b:true, r:ref)')
pn1 = ax2.plot(range(len(p9_data)),p9_data, label = "Y")
# pn1 = ax2.plot(range(len(p9_hat_data)),p9_hat_data, 'k:', label = "Y_hat")
pn1 = ax2.plot(range(len(py_r_data)),py_r_data, 'r:', label = "Y_ref")
ax2.legend(loc = "best")

ax3 = fig5.add_subplot(4, 3, 3)
ax3.set_xlabel('Time Step')
ax3.set_ylabel('Z')
ax3.set_title('Z vs Time Step (b:true, r:ref)')
pn1 = ax3.plot(range(len(p11_data)),p11_data, label = "Z")
# pn1 = ax3.plot(range(len(p11_hat_data)),p11_hat_data, 'k:', label = "Z_hat")
pn1 = ax3.plot(range(len(pz_r_data)),pz_r_data, 'r:', label = "Z_ref")
ax3.legend(loc = "best")

ax4 = fig5.add_subplot(4, 3, 4)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('Phi')
ax4.set_title('State Plot of Phi (b:true, r:ref)')
pn1 = ax4.plot(range(len(p1_data)),p1_data, label = "Phi")
pn1 = ax4.plot(range(len(px1d_data)),px1d_data, 'r:', label = "Phi_d")
ax4.legend(loc = "best")

ax5 = fig5.add_subplot(4, 3, 5)
ax5.set_xlabel('Time Step')
ax5.set_ylabel('Theta')
ax5.set_title('State Plot of Theta (b:true, r:ref)')
pn1 = ax5.plot(range(len(p3_data)),p3_data, label = "Theta")
pn1 = ax5.plot(range(len(px3d_data)),px3d_data, 'r:', label = "Theta_d")
ax5.legend(loc = "best")

ax6 = fig5.add_subplot(4, 3, 6)
ax6.set_xlabel('Time Step')
ax6.set_ylabel('Psi')
ax6.set_title('State Plot of Psi (b:true, r:ref)')
pn1 = ax6.plot(range(len(p5_data)),p5_data, label = "Psi")
# pn1 = ax6.plot(range(len(p5_hat_data)),p5_hat_data, 'k:', label = "Psi_hat")
pn1 = ax6.plot(range(len(ppsi_r_data)),ppsi_r_data, 'r:', label = "Psi_ref")
ax6.legend(loc = "best")

## Payload Inputs

In [ ]:
fig6 = plt.figure(figsize=(15,20))
ax1 = fig6.add_subplot(4, 2, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('U1')
ax1.set_title('U1 vs Time Step')
pn1 = ax1.plot(range(len(pU1_data)),pU1_data, label = "U1")
ax1.legend(loc = "best")

ax2 = fig6.add_subplot(4, 2, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('U2')
ax2.set_title('U2 vs Time Step')
pn1 = ax2.plot(range(len(pU2_data)),pU2_data, label = "U2")
ax2.legend(loc = "best")

ax3 = fig6.add_subplot(4, 2, 3)
ax3.set_xlabel('Time Step')
ax3.set_ylabel('U3')
ax3.set_title('U3 vs Time Step')
pn1 = ax3.plot(range(len(pU3_data)),pU3_data, label = "U3")
ax3.legend(loc = "best")

ax4 = fig6.add_subplot(4, 2, 4)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('U4')
ax4.set_title('U4 vs Time Step')
pn1 = ax4.plot(range(len(pU4_data)),pU4_data, label = "U4")
ax4.legend(loc = "best")

ax5 = fig6.add_subplot(4, 2, 5)
ax5.set_xlabel('Time Step')
ax5.set_ylabel('w1')
ax5.set_title('w1 vs Time Step')
pn1 = ax5.plot(range(len(pw1_data)),pw1_data, label = "w1")
ax5.legend(loc = "best")

ax6 = fig6.add_subplot(4, 2, 6)
ax6.set_xlabel('Time Step')
ax6.set_ylabel('w2')
ax6.set_title('w2 vs Time Step')
pn1 = ax6.plot(range(len(pw2_data)),pw2_data, label = "w2")
ax6.legend(loc = "best")

ax7 = fig6.add_subplot(4, 2, 7)
ax7.set_xlabel('Time Step')
ax7.set_ylabel('w3')
ax7.set_title('w3 vs Time Step')
pn1 = ax7.plot(range(len(pw3_data)),pw3_data, label = "w3")
ax7.legend(loc = "best")

ax8 = fig6.add_subplot(4, 2, 8)
ax8.set_xlabel('Time Step')
ax8.set_ylabel('w4')
ax8.set_title('w4 vs Time Step')
pn1 = ax8.plot(range(len(pw4_data)),pw4_data, label = "w4")
ax8.legend(loc = "best")

## Payloads Estimated Values

In [ ]:
fig6 = plt.figure(figsize=(17,17))
ax1 = fig6.add_subplot(3, 4, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('a1')
ax1.set_title('a1 vs Time Step')
pn1 = ax1.plot(range(len(pa1_hat_data)),pa1_hat_data, label = "a1_hat")
pn1 = ax1.plot(range(len(a1_data)),a1_data, 'r:', label = "a1")
ax1.legend(loc = "best")

ax2 = fig6.add_subplot(3, 4, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('a2')
ax2.set_title('a2 vs Time Step')
pn1 = ax2.plot(range(len(pa2_hat_data)),pa2_hat_data, label = "a2_hat")
pn1 = ax2.plot(range(len(a2_data)),a2_data, 'r:', label = "a2")
ax2.legend(loc = "best")

ax3 = fig6.add_subplot(3, 4, 3)
ax3.set_xlabel('Time Step')
ax3.set_ylabel('a3')
ax3.set_title('a3 vs Time Step')
pn1 = ax3.plot(range(len(pa3_hat_data)),pa3_hat_data, label = "a3_hat")
pn1 = ax3.plot(range(len(a3_data)),a3_data, 'r:', label = "a3")
ax3.legend(loc = "best")

ax4 = fig6.add_subplot(3, 4, 4)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('a4')
ax4.set_title('a4 vs Time Step')
pn1 = ax4.plot(range(len(pa4_hat_data)),pa4_hat_data, label = "a4_hat")
pn1 = ax4.plot(range(len(a4_data)),a4_data, 'r:', label = "a4")
ax4.legend(loc = "best")

ax5 = fig6.add_subplot(3, 4, 5)
ax5.set_xlabel('Time Step')
ax5.set_ylabel('a5')
ax5.set_title('a5 vs Time Step')
pn1 = ax5.plot(range(len(pa5_hat_data)),pa5_hat_data, label = "a5_hat")
pn1 = ax5.plot(range(len(a5_data)),a5_data, 'r:', label = "a5")
ax5.legend(loc = "best")

ax6 = fig6.add_subplot(3, 4, 6)
ax6.set_xlabel('Time Step')
ax6.set_ylabel('a6')
ax6.set_title('a6 vs Time Step')
pn1 = ax6.plot(range(len(pa6_hat_data)),pa6_hat_data, label = "a6_hat")
pn1 = ax6.plot(range(len(a6_data)),a6_data, 'r:', label = "a6")
ax6.legend(loc = "best")

ax7 = fig6.add_subplot(3, 4, 7)
ax7.set_xlabel('Time Step')
ax7.set_ylabel('a7')
ax7.set_title('a7 vs Time Step')
pn1 = ax7.plot(range(len(pa7_hat_data)),pa7_hat_data, label = "a7_hat")
pn1 = ax7.plot(range(len(a7_data)),a7_data, 'r:', label = "a7")
ax7.legend(loc = "best")

ax8 = fig6.add_subplot(3, 4, 8)
ax8.set_xlabel('Time Step')
ax8.set_ylabel('a8')
ax8.set_title('a8 vs Time Step')
pn1 = ax8.plot(range(len(pa8_hat_data)),pa8_hat_data, label = "a8_hat")
pn1 = ax8.plot(range(len(a8_data)),a8_data, 'r:', label = "a8")
ax8.legend(loc = "best")

ax9 = fig6.add_subplot(3, 4, 9)
ax9.set_xlabel('Time Step')
ax9.set_ylabel('a9')
ax9.set_title('a9 vs Time Step')
pn1 = ax9.plot(range(len(pa9_hat_data)),pa9_hat_data, label = "a9_hat")
pn1 = ax9.plot(range(len(a9_data)),a9_data, 'r:', label = "a9")
ax9.legend(loc = "best")

ax10 = fig6.add_subplot(3, 4, 10)
ax10.set_xlabel('Time Step')
ax10.set_ylabel('a10')
ax10.set_title('a10 vs Time Step')
pn1 = ax10.plot(range(len(pa10_hat_data)),pa10_hat_data, label = "a10_hat")
pn1 = ax10.plot(range(len(a10_data)),a10_data, 'r:', label = "a10")
ax10.legend(loc = "best")

ax11 = fig6.add_subplot(3, 4, 11)
ax11.set_xlabel('Time Step')
ax11.set_ylabel('a11')
ax11.set_title('a11 vs Time Step')
pn1 = ax11.plot(range(len(pa11_hat_data)),pa11_hat_data, label = "a11_hat")
pn1 = ax11.plot(range(len(a11_data)),a11_data, 'r:', label = "a11")
ax11.legend(loc = "best")

## Payload Localization

In [ ]:
fig7 = plt.figure(figsize=(15,20))
ax1 = fig7.add_subplot(4, 2, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('X')
ax1.set_title('X vs Time Step (b:true, r:ref)')
pn1 = ax1.plot(range(len(p7_data)),p7_data, label = "X")
pn1 = ax1.plot(range(len(p7_hat_data)),p7_hat_data, 'r:', label = "X_hat")
ax1.legend(loc = "best")

ax2 = fig7.add_subplot(4, 2, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('Y')
ax2.set_title('Y vs Time Step (b:true, r:ref)')
pn1 = ax2.plot(range(len(p9_data)),p9_data, label = "Y")
pn1 = ax2.plot(range(len(p9_hat_data)),p9_hat_data, 'r:', label = "Y_hat")
ax2.legend(loc = "best")

ax3 = fig7.add_subplot(4, 2, 3)
ax3.set_xlabel('Time Step')
ax3.set_ylabel('Z')
ax3.set_title('Z vs Time Step (b:true, r:ref)')
pn1 = ax3.plot(range(len(p11_data)),p11_data, label = "Z")
pn1 = ax3.plot(range(len(p11_hat_data)),p11_hat_data, 'r:', label = "Z_hat")
ax3.legend(loc = "best")

ax6 = fig7.add_subplot(4, 2, 4)
ax6.set_xlabel('Time Step')
ax6.set_ylabel('Psi')
ax6.set_title('State Plot of Psi (b:true, r:ref)')
pn1 = ax6.plot(range(len(p5_data)),p5_data, label = "Psi")
pn1 = ax6.plot(range(len(p5_hat_data)),p5_hat_data, 'r:', label = "Psi_hat")
ax6.legend(loc = "best")

### Covariance Plots

In [ ]:
fig8 = plt.figure(figsize=(10,10))

ax1 = fig8.add_subplot(2, 2, 1)
ax1.set_xlabel('Time Step'), 
ax1.set_ylabel('Psi')
ax1.set_title('Error for Psi')
P_EKF_pn1 = ax1.plot(range(len(P_ekf_p5)),P_ekf_p5, 'b')
error_pn1 = ax1.plot(range(len(p5_err)),p5_err, 'r')
NP_EKF_pn1 = ax1.plot(range(len(NP_ekf_p5)),NP_ekf_p5, 'b')

ax2 = fig8.add_subplot(2, 2, 2)
ax2.set_xlabel('Time Step'), 
ax2.set_ylabel('X')
ax2.set_title('Error for X')
P_EKF_pn1 = ax2.plot(range(len(P_ekf_p7)),P_ekf_p7, 'b')
error_pn1 = ax2.plot(range(len(p7_err)),p7_err, 'r')
NP_EKF_pn1 = ax2.plot(range(len(NP_ekf_p7)),NP_ekf_p7, 'b')

ax3 = fig8.add_subplot(2, 2, 3)
ax3.set_xlabel('Time Step'), 
ax3.set_ylabel('Y')
ax3.set_title('Error for Y')
P_EKF_pn1 = ax3.plot(range(len(P_ekf_p9)),P_ekf_p9, 'b')
error_pn1 = ax3.plot(range(len(p9_err)),p9_err, 'r')
NP_EKF_pn1 = ax3.plot(range(len(NP_ekf_p9)),NP_ekf_p9, 'b')

ax4 = fig8.add_subplot(2, 2, 4)
ax4.set_xlabel('Time Step'), 
ax4.set_ylabel('Z')
ax4.set_title('Error for Z')
P_EKF_pn1 = ax4.plot(range(len(P_ekf_p11)),P_ekf_p11, 'b')
error_pn1 = ax4.plot(range(len(p11_err)),p11_err, 'r')
NP_EKF_pn1 = ax4.plot(range(len(NP_ekf_p11)),NP_ekf_p11, 'b')